In [1]:
import pandas as pd
import re
import os
from pathlib import Path

def clean_text_for_rnn(text):
    """Clean and prepare text for RNN training"""
    # Remove extra whitespace and normalize
    text = ' '.join(text.split())
    
    # Remove special characters but keep punctuation for style
    text = re.sub(r'[^\w\s\.\,\!\?\;\:\-\'\"]', ' ', text)
    
    # Remove multiple spaces
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

def create_author_dataset_record(author, title, text, text_type="letters"):
    """Create a single record for RNN training dataset"""
    
    cleaned_text = clean_text_for_rnn(text)
    
    # Calculate text statistics for quality control
    word_count = len(cleaned_text.split())
    char_count = len(cleaned_text)
    
    return {
        'author': author,
        'title': title,
        'text': cleaned_text,
        'text_type': text_type,
        'word_count': word_count,
        'char_count': char_count,
        'suitable_for_training': word_count >= 100  # Minimum words for training
    }

def parse_filename(filename):
    """Extract author and title from filename"""
    # Remove .txt extension
    name_without_ext = filename.replace('.txt', '')
    
    # Split by '___' to get author and title
    if '___' in name_without_ext:
        parts = name_without_ext.split('___')
        author = parts[0].strip()
        title = parts[1].strip() if len(parts) > 1 else "Unknown Title"
    else:
        # Fallback if format is different
        author = name_without_ext
        title = "Unknown Title"
    
    return author, title

def process_gutenberg_files():
    """Process all files in the Gutenberg directory structure"""
    
    # Define the path to the text files
    txt_path = Path('Gutenberg_original/Gutenberg/txt')
    
    # Check if directory exists
    if not txt_path.exists():
        print(f"Directory {txt_path} does not exist!")
        return None
    
    all_records = []
    processed_files = 0
    skipped_files = 0
    
    # Process all .txt files in the directory
    for file_path in txt_path.glob('*.txt'):
        try:
            print(f"Processing: {file_path.name}")
            
            # Parse author and title from filename
            author, title = parse_filename(file_path.name)
            
            # Read the file content
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
                text_content = file.read()
            
            # Skip if file is too small
            if len(text_content.strip()) < 100:
                print(f"  Skipped: File too small ({len(text_content)} chars)")
                skipped_files += 1
                continue
            
            # Create dataset record
            record = create_author_dataset_record(
                author=author,
                title=title,
                text=text_content,
                text_type="letters"  # You can modify this based on content analysis
            )
            
            # Only add if suitable for training
            if record['suitable_for_training']:
                # Remove the suitable_for_training field for final dataset
                final_record = {
                    'author': record['author'],
                    'title': record['title'],
                    'text': record['text'],
                    'text_type': record['text_type'],
                    'word_count': record['word_count']
                }
                all_records.append(final_record)
                processed_files += 1
                print(f"  ✓ Added: {author} - {title} ({record['word_count']} words)")
            else:
                print(f"  Skipped: Not suitable for training ({record['word_count']} words)")
                skipped_files += 1
                
        except Exception as e:
            print(f"  Error processing {file_path.name}: {str(e)}")
            skipped_files += 1
            continue
    
    print(f"\n=== PROCESSING SUMMARY ===")
    print(f"Files processed successfully: {processed_files}")
    print(f"Files skipped: {skipped_files}")
    print(f"Total records created: {len(all_records)}")
    
    if all_records:
        # Create DataFrame
        df = pd.DataFrame(all_records)
        
        # Display summary
        print(f"\n=== DATASET SUMMARY ===")
        print(f"Total authors: {df['author'].nunique()}")
        print(f"Total records: {len(df)}")
        print(f"Average word count: {df['word_count'].mean():.1f}")
        print(f"Min word count: {df['word_count'].min()}")
        print(f"Max word count: {df['word_count'].max()}")
        
        print(f"\n=== AUTHORS IN DATASET ===")
        author_counts = df['author'].value_counts()
        for author, count in author_counts.items():
            print(f"{author}: {count} text(s)")
        
        # Save to CSV
        output_filename = 'author_identification_dataset.csv'
        df.to_csv(output_filename, index=False)
        print(f"\n✓ Dataset saved as '{output_filename}'")
        
        # Display first few rows
        print(f"\n=== SAMPLE RECORDS ===")
        print(df.head())
        
        return df
    else:
        print("No records were created!")
        return None

# Run the processing
print("Starting to process Gutenberg files...")
dataset = process_gutenberg_files()

if dataset is not None:
    print(f"\n=== FINAL DATASET INFO ===")
    print(dataset.info())
else:
    print("Failed to create dataset!")

Starting to process Gutenberg files...
Processing: Abraham Lincoln___Lincoln Letters.txt
  ✓ Added: Abraham Lincoln - Lincoln Letters (1065 words)
Processing: Abraham Lincoln___Lincoln's First Inaugural Address.txt
  ✓ Added: Abraham Lincoln - Lincoln's First Inaugural Address (3626 words)
Processing: Abraham Lincoln___Lincoln's Gettysburg Address, given November 19, 1863.txt
  ✓ Added: Abraham Lincoln - Lincoln's Gettysburg Address, given November 19, 1863 (299 words)
Processing: Abraham Lincoln___Lincoln's Inaugurals, Addresses and Letters (Selections).txt
  ✓ Added: Abraham Lincoln - Lincoln's Inaugurals, Addresses and Letters (Selections) (43649 words)
Processing: Abraham Lincoln___Lincoln's Second Inaugural Address.txt
  ✓ Added: Abraham Lincoln - Lincoln's Second Inaugural Address (703 words)
Processing: Abraham Lincoln___Speeches and Letters of Abraham Lincoln, 1832-1865.txt
  ✓ Added: Abraham Lincoln - Speeches and Letters of Abraham Lincoln, 1832-1865 (90170 words)
Processing:

In [3]:
import pandas as pd

author_identify = pd.read_csv('author_identification_dataset.csv')

author_identify.head()

,author,title,text,text_type,word_count
0,Abraham Lincoln,Lincoln Letters,LINCOLN LETTERS By Abraham Lincoln Published b...,letters,1065
1,Abraham Lincoln,Lincoln's First Inaugural Address,"Lincoln's First Inaugural Address March 4, 186...",letters,3626
2,Abraham Lincoln,"Lincoln's Gettysburg Address, given November 1...","Lincoln's Gettysburg Address, given November 1...",letters,299
3,Abraham Lincoln,"Lincoln's Inaugurals, Addresses and Letters (S...",Longman's English Classics LINCOLN'S INAUGURAL...,letters,43649
4,Abraham Lincoln,Lincoln's Second Inaugural Address,"Lincoln's Second Inaugural Address March 4, 18...",letters,703


In [4]:
import pandas as pd

# Load dataset
author_identify = pd.read_csv('author_identification_dataset.csv')

# 1. Number of unique authors
num_authors = author_identify['author'].nunique()

# 2. Number of unique titles (books)
num_books = author_identify['title'].nunique()

# 3. Average number of characters per book
# Calculate length of each text and then average (grouping by title in case of duplicates)
book_char_lengths = author_identify.groupby('title')['text'].apply(lambda x: x.iloc[0]).apply(len)
avg_chars_per_book = book_char_lengths.mean()

print("Number of unique authors:", num_authors)
print("Number of unique books:", num_books)
print("Average number of characters per book:", round(avg_chars_per_book, 2))

Number of unique authors: 142
Number of unique books: 3022
Average number of characters per book: 386923.94


In [7]:
import pandas as pd

# Load dataset
author_identify = pd.read_csv('author_identification_dataset.csv')

# Get all unique titles
unique_titles = author_identify['title'].unique()

# Convert to list if you want to handle them outside pandas
unique_titles_list = list(unique_titles)

# Number of items per section
chunk_size = 600

# Split into chunks of 600 and print
for i in range(0, len(unique_titles_list), chunk_size):
    chunk = unique_titles_list[i:i+chunk_size]
    print(f"\nSection {i//chunk_size + 1}:")
    print("Unique titles = [")
    for title in chunk:
        print(f"    '{title}',")
    print("]")


Section 1:
Unique titles = [
    'Lincoln Letters',
    'Lincoln's First Inaugural Address',
    'Lincoln's Gettysburg Address, given November 19, 1863',
    'Lincoln's Inaugurals, Addresses and Letters (Selections)',
    'Lincoln's Second Inaugural Address',
    'Speeches and Letters of Abraham Lincoln, 1832-1865',
    'State of the Union Addresses',
    'The Emancipation Proclamation',
    'The Life and Public Service of General Zachary Taylor_ An Address',
    'The Writings of Abraham Lincoln, Volume 1_ 1832-1843',
    'The Writings of Abraham Lincoln, Volume 2_ 1843-1858',
    'The Writings of Abraham Lincoln, Volume 3',
    'The Writings of Abraham Lincoln, Volume 4',
    'The Writings of Abraham Lincoln, Volume 5_ 1858-1862',
    'The Writings of Abraham Lincoln, Volume 6_ 1862-1863',
    'The Writings of Abraham Lincoln, Volume 7_ 1863-1865',
    'The Mysterious Affair at Styles',
    'The Secret Adversary',
    'Relativity_ The Special and General Theory',
    'Sidelights on R

In [ ]:
genre_map = {
    "Lincoln Letters": "History/Politics, Non-Fiction",
    "Lincoln's First Inaugural Address": "History/Politics, Speech/Non-Fiction",
    "Lincoln's Gettysburg Address, given November 19, 1863": "History/Politics, Speech/Non-Fiction",
    "Lincoln's Inaugurals, Addresses and Letters (Selections)": "History/Politics, Non-Fiction",
    "Lincoln's Second Inaugural Address": "History/Politics, Speech/Non-Fiction",
    "Speeches and Letters of Abraham Lincoln, 1832-1865": "History/Politics, Non-Fiction",
    "State of the Union Addresses": "History/Politics, Speech/Non-Fiction",
    "The Emancipation Proclamation": "History/Politics, Non-Fiction",
    "The Life and Public Service of General Zachary Taylor_ An Address": "History/Biography, Non-Fiction",
    "The Writings of Abraham Lincoln, Volume 1_ 1832-1843": "History/Politics, Non-Fiction",
    "The Writings of Abraham Lincoln, Volume 2_ 1843-1858": "History/Politics, Non-Fiction",
    "The Writings of Abraham Lincoln, Volume 3": "History/Politics, Non-Fiction",
    "The Writings of Abraham Lincoln, Volume 4": "History/Politics, Non-Fiction",
    "The Writings of Abraham Lincoln, Volume 5_ 1858-1862": "History/Politics, Non-Fiction",
    "The Writings of Abraham Lincoln, Volume 6_ 1862-1863": "History/Politics, Non-Fiction",
    "The Writings of Abraham Lincoln, Volume 7_ 1863-1865": "History/Politics, Non-Fiction",
    "The Mysterious Affair at Styles": "Mystery/Crime, Fiction",
    "The Secret Adversary": "Mystery/Adventure, Fiction",
    "Relativity_ The Special and General Theory": "Science/Physics, Non-Fiction",
    "Sidelights on Relativity": "Science/Physics, Non-Fiction",
    "Crome Yellow": "Literature/Satire, Fiction",
    "Mortal Coils": "Literature/Short Stories, Fiction",
    "The Defeat of Youth and Other Poems": "Poetry, Fiction",
    "An Essay on Criticism": "Literature/Essay, Non-Fiction",
    "Essay on Man": "Poetry/Philosophy, Non-Fiction",
    "The Poetical Works of Alexander Pope, Volume 1": "Poetry, Fiction",
    "The Poetical Works of Alexander Pope, Volume 2": "Poetry, Fiction",
    "The Rape of the Lock and Other Poems": "Poetry/Satire, Fiction",
    "The Works of Alexander Pope, Volume 1": "Literature/Poetry, Fiction",
    "Contributions to the Theory of Natural Selection": "Science/Evolution, Non-Fiction",
    "Darwinism": "Science/Evolution, Non-Fiction",
    "Is Mars Habitable_": "Science/Astronomy, Non-Fiction",
    "Island Life": "Science/Biology, Non-Fiction",
    "The Malay Archipelago, Volume 1": "Science/Travel, Non-Fiction",
    "The Malay Archipelago, Volume 2": "Science/Travel, Non-Fiction",
    "A Cynic Looks at Life": "Literature/Essay, Non-Fiction",
    "A Son of the Gods, and A Horseman in the Sky": "Literature/Short Stories, Fiction",
    "An Occurrence at Owl Creek Bridge": "Literature/Short Story, Fiction",
    "Black Beetles in Amber": "Poetry/Satire, Fiction",
    "Can Such Things Be_": "Horror/Short Stories, Fiction",
    "Cobwebs From an Empty Skull": "Humor/Satire, Fiction",
    "Fantastic Fables": "Literature/Fables, Fiction",
    "Present at a Hanging and Other Ghost Stories": "Horror/Short Stories, Fiction",
    "Shapes of Clay": "Poetry, Fiction",
    "The Collected Works of Ambrose Bierce, Volume 1": "Literature/Collection, Fiction",
    "The Collected Works of Ambrose Bierce, Volume 2": "Literature/Collection, Fiction",
    "The Collected Works of Ambrose Bierce, Volume 8": "Literature/Collection, Fiction",
    "The Damned Thing": "Horror/Short Story, Fiction",
    "The Devil's Dictionary": "Humor/Satire, Non-Fiction",
    "The Fiend's Delight": "Humor/Satire, Fiction",
    "The Parenticide Club": "Humor/Short Stories, Fiction",
    "The Shadow On The Dial, and Other Essays": "Literature/Essays, Non-Fiction",
    "Write It Right": "Language/Writing, Non-Fiction",
    "A Collection of Ballads": "Poetry/Folklore, Fiction",
    "A Monk of Fife": "Historical/Adventure, Fiction",
    "A Short History of Scotland": "History, Non-Fiction",
    "Adventures among Books": "Literature/Essays, Non-Fiction",
    "Alfred Tennyson": "Biography/Literature, Non-Fiction",
    "Andrew Lang's Introduction to The Compleat Angler": "Literature/Essay, Non-Fiction",
    "Angling Sketches": "Sports/Outdoor, Non-Fiction",
    "Ballads in Blue China and Verses and Translations": "Poetry, Fiction",
    "Ban and Arriere Ban": "Poetry, Fiction",
    "Books and Bookmen": "Literature/Essays, Non-Fiction",
    "Cock Lane and Common-Sense": "Folklore/Supernatural, Non-Fiction",
    "Custom and Myth": "Mythology/Anthropology, Non-Fiction",
    "Essays in Little": "Literature/Essays, Non-Fiction",
    "Grass of Parnassus": "Poetry, Fiction",
    "He": "Humor/Fantasy, Fiction",
    "Helen of Troy": "Poetry/Mythology, Fiction",
    "Historical Mysteries": "History/Mystery, Non-Fiction",
    "Homer and His Age": "History/Literature, Non-Fiction",
    "How to Fail in Literature": "Literature/Humor, Non-Fiction",
    "In the Wrong Paradise": "Fantasy/Short Stories, Fiction",
    "James VI and the Gowrie Mystery": "History/Mystery, Non-Fiction",
    "John Knox and the Reformation": "History/Religion, Non-Fiction",
    "Letters on Literature": "Literature/Essays, Non-Fiction",
    "Letters to Dead Authors": "Literature/Essays, Non-Fiction",
    "Lost Leaders": "Literature/Essays, Non-Fiction",
    "Modern Mythology": "Mythology/Anthropology, Non-Fiction",
    "Much Darker Days": "Humor/Satire, Fiction",
    "Myth, Ritual And Religion, Vol. 2 (of 2)": "Mythology/Religion, Non-Fiction",
    "Myth, Ritual, and Religion, Vol. 1": "Mythology/Religion, Non-Fiction",
    "New Collected Rhymes": "Poetry, Fiction",
    "Old Friends - Essays in Epistolary Parody": "Literature/Humor, Fiction",
    "Oxford": "History/Travel, Non-Fiction",
    "Pickle the Spy": "History/Biography, Non-Fiction",
    "Prince Prigio From _His Own Fairy Book_": "Fantasy/Children's, Fiction",
    "Prince Prigio": "Fantasy/Children's, Fiction",
    "Prince Ricardo of Pantouflia": "Fantasy/Adventure, Fiction",
    "Rhymes a la Mode": "Poetry, Fiction",
    "Shakespeare, Bacon and the Great Unknown": "Literature/Criticism, Non-Fiction",
    "Sir Walter Scott and the Border Minstrelsy": "Literature/Criticism, Non-Fiction",
    "Tales of Troy and Greece": "Mythology/Adventure, Fiction",
    "The Arabian Nights Entertainments": "Fantasy/Folklore, Fiction",
    "The Blue Fairy Book": "Children's/Fairy Tales, Fiction",
    "The Book of Dreams and Ghosts": "Supernatural/Folklore, Non-Fiction",
    "The Brown Fairy Book": "Children's/Fairy Tales, Fiction",
    "The Clyde Mystery": "History/Mystery, Non-Fiction",
    "The Crimson Fairy Book": "Children's/Fairy Tales, Fiction",
    "The Disentanglers": "Mystery/Adventure, Fiction",
    "The Gold Of Fairnilee": "Fantasy/Children's, Fiction",
    "The Grey Fairy Book": "Children's/Fairy Tales, Fiction",
    "The Library": "Literature/Essay, Non-Fiction",
    "The Lilac Fairy Book": "Children's/Fairy Tales, Fiction",
    "The Making of Religion": "Religion/Anthropology, Non-Fiction",
    "The Mark Of Cain": "Mystery/Crime, Fiction",
    "The Orange Fairy Book": "Children's/Fairy Tales, Fiction",
    "The Pink Fairy Book": "Children's/Fairy Tales, Fiction",
    "The Puzzle of Dickens's Last Plot": "Literature/Criticism, Non-Fiction",
    "The Red Fairy Book": "Children's/Fairy Tales, Fiction",
    "The Valet's Tragedy and Other Stories": "Mystery/Short Stories, Fiction",
    "The Violet Fairy Book": "Children's/Fairy Tales, Fiction",
    "The Yellow Fairy Book": "Children's/Fairy Tales, Fiction",
    "A Ride Across Palestine": "Adventure/Short Story, Fiction",
    "Aaron Trow": "Literature/Short Story, Fiction",
    "An Eye for an Eye": "Literature/Novel, Fiction",
    "An Old Man's Love": "Literature/Romance, Fiction",
    "An Unprotected Female at the Pyramids": "Literature/Short Story, Fiction",
    "Autobiography of Anthony Trollope": "Biography, Non-Fiction",
    "Ayala's Angel": "Literature/Romance, Fiction",
    "Barchester Towers": "Literature/Novel, Fiction",
    "Can You Forgive Her_": "Literature/Novel, Fiction",
    "Castle Richmond": "Literature/Novel, Fiction",
    "Cousin Henry": "Literature/Novel, Fiction",
    "Doctor Thorne": "Literature/Novel, Fiction",
    "Dr. Wortle's School": "Literature/Novel, Fiction",
    "Framley Parsonage": "Literature/Novel, Fiction",
    "George Walker At Suez": "Literature/Short Story, Fiction",
    "Harry Heathcote of Gangoil": "Adventure/Novel, Fiction",
    "He Knew He Was Right": "Literature/Novel, Fiction",
    "Hunting Sketches": "Sports/Humor, Non-Fiction",
    "Is He Popenjoy_": "Literature/Novel, Fiction",
    "John Bull on the Guadalquivir from Tales from all Countries": "Literature/Short Story, Fiction",
    "John Caldigate": "Literature/Novel, Fiction",
    "Kept in the Dark": "Literature/Novel, Fiction",
    "La Mere Bauche from Tales of All Countries": "Literature/Short Story, Fiction",
    "La Vendée": "Historical/Novel, Fiction",
    "Lady Anna": "Literature/Novel, Fiction",
    "Life of Cicero Volume One": "Biography/History, Non-Fiction",
    "Life of Cicero Volume Two": "Biography/History, Non-Fiction",
    "Linda Tressel": "Literature/Novel, Fiction",
    "Marion Fay": "Literature/Novel, Fiction",
    "Miss Mackenzie": "Literature/Novel, Fiction",
    "Miss Sarah Jack, of Spanish Town, Jamaica": "Literature/Short Story, Fiction",
    "Mr. Scarborough's Family": "Literature/Novel, Fiction",
    "Mrs. General Talboys": "Literature/Short Story, Fiction",
    "Nina Balatka": "Literature/Novel, Fiction",
    "North America Volume 1": "Travel/History, Non-Fiction",
    "North America Volume 2": "Travel/History, Non-Fiction",
    "Orley Farm": "Literature/Novel, Fiction",
    "Phineas Finn": "Literature/Novel, Fiction",
    "Phineas Redux": "Literature/Novel, Fiction",
    "Rachel Ray": "Literature/Novel, Fiction",
    "Ralph the Heir": "Literature/Novel, Fiction",
    "Returning Home": "Literature/Short Story, Fiction",
    "Sir Harry Hotspur of Humblethwaite": "Literature/Novel, Fiction",
    "Thackeray": "Biography/Literature, Non-Fiction",
    "The American Senator": "Literature/Novel, Fiction",
    "The Belton Estate": "Literature/Novel, Fiction",
    "The Bertrams": "Literature/Novel, Fiction",
    "The Chateau of Prince Polignac": "Literature/Short Story, Fiction",
    "The Claverings": "Literature/Novel, Fiction",
    "The Courtship of Susan Bell": "Literature/Short Story, Fiction",
    "The Duke's Children": "Literature/Novel, Fiction",
    "The Eustace Diamonds": "Literature/Novel, Fiction",
    "The Fixed Period": "Literature/Novel, Fiction",
    "The Golden Lion of Granpere": "Literature/Novel, Fiction",
    "The House of Heine Brothers, in Munich": "Literature/Short Story, Fiction",
    "The Kellys and the O'Kellys": "Literature/Novel, Fiction",
    "The Landleaguers": "Literature/Novel, Fiction",
    "The Last Chronicle of Barset": "Literature/Novel, Fiction",
    "The Macdermots of Ballycloran": "Literature/Novel, Fiction",
    "The Man Who Kept His Money In A Box": "Literature/Short Story, Fiction",
    "The Mistletoe Bough": "Literature/Short Story, Fiction",
    "The O'Conors of Castle Conor from Tales from all Countries": "Literature/Short Story, Fiction",
    "The Parson's Daughter of Oxney Colne": "Literature/Short Story, Fiction",
    "The Prime Minister": "Literature/Novel, Fiction",
    "The Relics of General Chasse": "Literature/Short Story, Fiction",
    "The Small House at Allington": "Literature/Novel, Fiction",
    "The Struggles of Brown, Jones, and Robinson": "Literature/Novel, Fiction",
    "The Three Clerks": "Literature/Novel, Fiction",
    "The Vicar of Bullhampton": "Literature/Novel, Fiction",
    "The Warden": "Literature/Novel, Fiction",
    "The Way We Live Now": "Literature/Novel, Fiction",
    "Turkey_ A Past and a Future": "History/Politics, Non-Fiction",
    "A Bride of the Plains": "Adventure/Romance, Fiction",
    "Castles in the Air": "Adventure/Romance, Fiction",
    "El Dorado": "Adventure/Historical, Fiction",
    "I Will Repay": "Adventure/Historical, Fiction",
    "Lord Tony's Wife": "Adventure/Historical, Fiction",
    "Petticoat Rule": "Adventure/Historical, Fiction",
    "The Bronze Eagle": "Adventure/Historical, Fiction",
    "The Elusive Pimpernel": "Adventure/Historical, Fiction",
    "The Heart of a Woman": "Adventure/Romance, Fiction",
    "The Laughing Cavalier": "Adventure/Historical, Fiction",
    "The League of the Scarlet Pimpernel": "Adventure/Short Stories, Fiction",
    "The Nest of the Sparrowhawk": "Adventure/Historical, Fiction",
    "The Old Man in the Corner": "Mystery/Detective, Fiction",
    "The Scarlet Pimpernel": "Adventure/Historical, Fiction",
    "The Tangled Skein": "Adventure/Historical, Fiction",
    "_Unto Caesar_": "Adventure/Historical, Fiction",
    "A Collection of Beatrix Potter Stories": "Children's/Short Stories, Fiction",
    "Cecily Parsley's Nursery Rhymes": "Children's/Poetry, Fiction",
    "The Great Big Treasury of Beatrix Potter": "Children's/Collection, Fiction",
    "The Story of Miss Moppet": "Children's/Story, Fiction",
    "The Tailor of Gloucester": "Children's/Story, Fiction",
    "The Tale Of Benjamin Bunny": "Children's/Story, Fiction",
    "The Tale of Ginger and Pickles": "Children's/Story, Fiction",
    "The Tale of Jemima Puddle-Duck": "Children's/Story, Fiction",
    "The Tale of Johnny Town-Mouse": "Children's/Story, Fiction",
    "The Tale of Mr. Jeremy Fisher": "Children's/Story, Fiction",
    "The Tale of Mr. Tod": "Children's/Story, Fiction",
    "The Tale of Mrs. Tiggy-Winkle": "Children's/Story, Fiction",
    "The Tale of Mrs. Tittlemouse": "Children's/Story, Fiction",
    "The Tale Of Peter Rabbit": "Children's/Story, Fiction",
    "The Tale of Samuel Whiskers": "Children's/Story, Fiction",
    "The Tale of Squirrel Nutkin": "Children's/Story, Fiction",
    "The Tale of the Flopsy Bunnies": "Children's/Story, Fiction",
    "The Tale of the Pie and the Patty Pan": "Children's/Story, Fiction",
    "The Tale of Timmy Tiptoes": "Children's/Story, Fiction",
    "The Tale of Tom Kitten": "Children's/Story, Fiction",
    "Alroy": "Historical/Novel, Fiction",
    "Coningsby": "Political/Novel, Fiction",
    "Count Alarcos": "Drama/Tragedy, Fiction",
    "Endymion": "Literature/Novel, Fiction",
    "Henrietta Temple": "Romance/Novel, Fiction",
    "Ixion In Heaven": "Fantasy/Satire, Fiction",
    "Lord George Bentinck": "Biography/Politics, Non-Fiction",
    "Lothair": "Literature/Novel, Fiction",
    "Sketches": "Literature/Short Stories, Fiction",
    "Sybil": "Political/Novel, Fiction",
    "Tancred": "Literature/Novel, Fiction",
    "The Infernal Marriage": "Fantasy/Satire, Fiction",
    "The Rise of Iskander": "Adventure/Novel, Fiction",
    "The Voyage of Captain Popanilla": "Adventure/Satire, Fiction",
    "The Young Duke": "Literature/Novel, Fiction",
    "Venetia": "Literature/Novel, Fiction",
    "Vivian Grey": "Literature/Novel, Fiction",
    "Autobiography of Benjamin Franklin, Version 1": "Biography, Non-Fiction",
    "Autobiography of Benjamin Franklin, Version 2": "Biography, Non-Fiction",
    "Autobiography of Benjamin Franklin, Version 3": "Biography, Non-Fiction",
    "Autobiography of Benjamin Franklin, Version 4": "Biography, Non-Fiction",
    "Mysticism and Logic and Other Essays": "Philosophy/Essays, Non-Fiction",
    "Our Knowledge of the External World as a Field for Scientific Method in Philosophy": "Philosophy/Science, Non-Fiction",
    "Political Ideals": "Philosophy/Politics, Non-Fiction",
    "Proposed Roads To Freedom": "Philosophy/Politics, Non-Fiction",
    "The Analysis of Mind": "Philosophy/Psychology, Non-Fiction",
    "The Practice and Theory of Bolshevism": "Philosophy/Politics, Non-Fiction",
    "The Problem of China": "History/Politics, Non-Fiction",
    "The Problems of Philosophy": "Philosophy, Non-Fiction",
    "Dracula's Guest": "Horror/Short Stories, Fiction",
    "Dracula": "Horror/Novel, Fiction",
    "The Jewel of Seven Stars": "Horror/Adventure, Fiction",
    "The Lady of the Shroud": "Horror/Adventure, Fiction",
    "The Lair of the White Worm": "Horror/Novel, Fiction",
    "The Man": "Horror/Adventure, Fiction",
    "A Drift from Redwood Camp": "Western/Short Story, Fiction",
    "A First Family of Tasajara": "Western/Novel, Fiction",
    "A Millionaire of Rough-and-Ready": "Western/Novel, Fiction",
    "A Phyllis of the Sierras": "Western/Novel, Fiction",
    "A Protegee of Jack Hamlin's and Other Stories": "Western/Short Stories, Fiction",
    "A Sappho of Green Springs": "Western/Short Stories, Fiction",
    "A Waif of the Plains": "Western/Adventure, Fiction",
    "A Ward of the Golden Gate": "Western/Novel, Fiction",
    "By Shore and Sedge": "Western/Short Stories, Fiction",
    "Clarence": "Western/Novel, Fiction",
    "Colonel Starbottle's Client and Other Stories": "Western/Short Stories, Fiction",
    "Complete Poetical Works of Bret Harte": "Poetry, Fiction",
    "Condensed Novels": "Humor/Satire, Fiction",
    "Cressy": "Western/Novel, Fiction",
    "Devil's Ford": "Western/Novel, Fiction",
    "Dickens in Camp": "Literature/Poetry, Fiction",
    "Drift from Two Shores": "Western/Short Stories, Fiction",
    "East and West": "Poetry, Fiction",
    "Excelsior": "Poetry, Fiction",
    "Flip, A California Romance": "Western/Romance, Fiction",
    "Found At Blazing Star": "Western/Short Story, Fiction",
    "From Sand Hill to Pine": "Western/Short Stories, Fiction",
    "Frontier Stories": "Western/Short Stories, Fiction",
    "Gabriel Conroy": "Western/Novel, Fiction",
    "Her Letter His Answer & Her Last Letter": "Poetry/Romance, Fiction",
    "In a Hollow of the Hills": "Western/Adventure, Fiction",
    "In the Carquinez Woods": "Western/Adventure, Fiction",
    "Jeff Briggs's Love Story": "Western/Romance, Fiction",
    "Legends and Tales": "Western/Short Stories, Fiction",
    "Maruja": "Western/Novel, Fiction",
    "Mr. Jack Hamlin's Mediation and Other Stories": "Western/Short Stories, Fiction",
    "Mrs. Skaggs's Husbands and Other Stories": "Western/Short Stories, Fiction",
    "New Burlesques": "Humor/Satire, Fiction",
    "On the Frontier": "Western/Short Stories, Fiction",
    "Openings in the Old Trail": "Western/Short Stories, Fiction",
    "Sally Dows and Other Stories": "Western/Short Stories, Fiction",
    "Salomy Jane": "Western/Short Story, Fiction",
    "Selected Stories": "Western/Short Stories, Fiction",
    "Snow-Bound at Eagle's": "Western/Adventure, Fiction",
    "Stories in Light and Shadow": "Western/Short Stories, Fiction",
    "Susy, A Story of the Plains": "Western/Novel, Fiction",
    "Tales of the Argonauts": "Western/Short Stories, Fiction",
    "Tales of Trail and Town": "Western/Short Stories, Fiction",
    "Tennessee's Partner": "Western/Short Story, Fiction",
    "Thankful Blossom": "Western/Novel, Fiction",
    "The Argonauts of North Liberty": "Western/Adventure, Fiction",
    "The Bell-Ringer of Angel's and Other Stories": "Western/Short Stories, Fiction",
    "The Crusade of the Excelsior": "Adventure/Novel, Fiction",
    "The Heritage of Dedlow Marsh and Other Tales": "Western/Short Stories, Fiction",
    "The Luck of Roaring Camp and Other Tales": "Western/Short Stories, Fiction",
    "The Queen of the Pirate Isle": "Children's/Adventure, Fiction",
    "The Story of a Mine": "Western/Adventure, Fiction",
    "The Three Partners": "Western/Adventure, Fiction",
    "The Twins of Table Mountain and Other Stories": "Western/Short Stories, Fiction",
    "Trent's Trust and Other Stories": "Western/Short Stories, Fiction",
    "Two Men of Sandy Bar": "Western/Drama, Fiction",
    "Under the Redwoods": "Western/Short Stories, Fiction",
    "Urban Sketches": "Literature/Sketches, Fiction",
    "A Monograph on the Sub-class Cirripedia (Volume 1 of 2)": "Science/Biology, Non-Fiction",
    "A Naturalist's Voyage Round the World": "Science/Travel, Non-Fiction",
    "Coral Reefs": "Science/Geology, Non-Fiction",
    "Geological Observations On South America": "Science/Geology, Non-Fiction",
    "Insectivorous Plants": "Science/Botany, Non-Fiction",
    "On the Origin of Species 1st Edition": "Science/Evolution, Non-Fiction",
    "On the Origin of Species 6th Edition": "Science/Evolution, Non-Fiction",
    "On the Origin of Species by Means of Natural Selection or the Preservation of Favoured Races in the Struggle for Life. (2nd edition)": "Science/Evolution, Non-Fiction",
    "The Autobiography of Charles Darwin": "Biography/Science, Non-Fiction",
    "The Descent of Man and Selection in Relation to Sex Volume I (1st edition)": "Science/Evolution, Non-Fiction",
    "The Descent of Man and Selection in Relation to Sex Volume II (1st Edition)": "Science/Evolution, Non-Fiction",
    "The Descent of Man and Selection in Relation to Sex": "Science/Evolution, Non-Fiction",
    "The Different Forms of Flowers on Plants of the Same Species": "Science/Botany, Non-Fiction",
    "The Effects of Cross & Self-Fertilisation in the Vegetable Kingdom": "Science/Botany, Non-Fiction",
    "The Expression of Emotion in Man and Animals": "Science/Psychology, Non-Fiction",
    "The Formation of Vegetable Mould through the action of worms with observations of their habits": "Science/Biology, Non-Fiction",
    "The Movements and Habits of Climbing Plants": "Science/Botany, Non-Fiction",
    "The Variation of Animals and Plants under Domestication Volume I": "Science/Biology, Non-Fiction",
    "The Variation of Animals and Plants under Domestication Volume II": "Science/Biology, Non-Fiction",
    "Volcanic Islands": "Science/Geology, Non-Fiction",
    "A Child's History of England": "History/Children's, Non-Fiction",
    "A Christmas Carol": "Fantasy/Christmas, Fiction",
    "A Message from the Sea": "Adventure/Short Story, Fiction",
    "A Tale of Two Cities": "Historical/Novel, Fiction",
    "American Notes for General Circulation": "Travel, Non-Fiction",
    "Barnaby Rudge": "Historical/Novel, Fiction",
    "Bleak House": "Literature/Novel, Fiction",
    "Captain Boldheart & the Latin-Grammar Master": "Children's/Adventure, Fiction",
    "Charles Dickens' Children Stories": "Children's/Short Stories, Fiction",
    "David Copperfield": "Literature/Novel, Fiction",
    "Doctor Marigold": "Literature/Short Story, Fiction",
    "Dombey and Son": "Literature/Novel, Fiction",
    "George Silverman's Explanation": "Literature/Short Story, Fiction",
    "Going into Society": "Literature/Short Story, Fiction",
    "Great Expectations": "Literature/Novel, Fiction",
    "Hard Times": "Literature/Novel, Fiction",
    "Holiday Romance": "Children's/Fantasy, Fiction",
    "Hunted Down": "Mystery/Short Stories, Fiction",
    "Life And Adventures Of Martin Chuzzlewit": "Literature/Novel, Fiction",
    "Little Dorrit": "Literature/Novel, Fiction",
    "Master Humphrey's Clock": "Literature/Short Stories, Fiction",
    "Miscellaneous Papers": "Literature/Essays, Non-Fiction",
    "Mrs. Lirriper's Legacy": "Literature/Short Story, Fiction",
    "Mrs. Lirriper's Lodgings": "Literature/Short Story, Fiction",
    "Mudfog and Other Sketches": "Humor/Sketches, Fiction",
    "Mugby Junction": "Literature/Short Stories, Fiction",
    "Nicholas Nickleby": "Literature/Novel, Fiction",
    "Oliver Twist": "Literature/Novel, Fiction",
    "Our Mutual Friend": "Literature/Novel, Fiction",
    "Pictures from Italy": "Travel, Non-Fiction",
    "Reprinted Pieces": "Literature/Essays, Non-Fiction",
    "Sketches by Boz": "Literature/Sketches, Fiction",
    "Sketches of Young Couples": "Humor/Sketches, Fiction",
    "Sketches of Young Gentlemen": "Humor/Sketches, Fiction",
    "Some Christmas Stories": "Fantasy/Short Stories, Fiction",
    "Somebody's Luggage": "Literature/Short Stories, Fiction",
    "Speeches_ Literary and Social": "Literature/Speeches, Non-Fiction",
    "Sunday Under Three Heads": "Literature/Essay, Non-Fiction",
    "The Battle of Life": "Literature/Novella, Fiction",
    "The Chimes": "Fantasy/Novella, Fiction",
    "The Cricket on the Hearth": "Fantasy/Novella, Fiction",
    "The Haunted Man and the Ghost's Bargin": "Fantasy/Novella, Fiction",
    "The Holly-Tree": "Literature/Short Story, Fiction",
    "The Lamplighter": "Literature/Short Story, Fiction",
    "The Lazy Tour of Two Idle Apprentices": "Literature/Travel, Fiction",
    "The Letters of Charles Dickens Volume 1": "Biography/Letters, Non-Fiction",
    "The Letters of Charles Dickens Volume 2": "Biography/Letters, Non-Fiction",
    "The Letters of Charles Dickens Volume 3": "Biography/Letters, Non-Fiction",
    "The Magic Fishbone": "Children's/Fantasy, Fiction",
    "The Mystery of Edwin Drood": "Mystery/Novel, Fiction",
    "The Old Curiosity Shop": "Literature/Novel, Fiction",
    "The Perils of Certain English Prisoners": "Adventure/Short Story, Fiction",
    "The Pickwick Papers": "Literature/Novel, Fiction",
    "The Poems and Verses of Charles Dickens": "Poetry, Fiction",
    "The Seven Poor Travellers": "Literature/Short Stories, Fiction",
    "The Trial of William Tinkling": "Children's/Humor, Fiction",
    "The Uncommercial Traveller": "Travel/Essays, Non-Fiction",
    "The Wreck of the Golden Mary": "Adventure/Short Story, Fiction",
    "Three Ghost Stories": "Horror/Short Stories, Fiction",
    "To be Read at Dusk": "Horror/Short Stories, Fiction",
    "Tom Tiddler's Ground": "Literature/Short Story, Fiction",
    "Alexandria and her Schools": "History/Philosophy, Non-Fiction",
    "All Saints' Day and Other Sermons": "Religion/Sermons, Non-Fiction",
    "Andromeda and Other Poems": "Poetry, Fiction",
    "At Last": "Travel/Science, Non-Fiction",
    "Daily Thoughts": "Religion/Inspirational, Non-Fiction",
    "David": "Religion/Sermons, Non-Fiction",
    "Discipline and Other Sermons": "Religion/Sermons, Non-Fiction",
    "Froude's History of England": "History/Criticism, Non-Fiction",
    "Glaucus; or The Wonders of the Shore": "Science/Nature, Non-Fiction",
    "Health and Education": "Health/Education, Non-Fiction",
    "Hereward, The Last of the English": "Historical/Novel, Fiction",
    "Historical Lectures and Essays": "History/Essays, Non-Fiction",
    "Hypatia": "Historical/Novel, Fiction",
    "Lectures Delivered in America in 1874": "History/Lectures, Non-Fiction",
    "Literary and General Lectures and Essays": "Literature/Essays, Non-Fiction",
    "Madam How and Lady Why": "Children's/Science, Non-Fiction",
    "Out of the Deep": "Religion/Sermons, Non-Fiction",
    "Phaethon": "Philosophy/Dialogues, Fiction",
    "Plays and Puritans": "Literature/Criticism, Non-Fiction",
    "Prose Idylls": "Literature/Essays, Non-Fiction",
    "Sanitary and Social Lectures and Essays": "Social Science/Essays, Non-Fiction",
    "Scientific Essays and Lectures": "Science/Lectures, Non-Fiction",
    "Sermons for the Times": "Religion/Sermons, Non-Fiction",
    "Sermons on National Subjects": "Religion/Sermons, Non-Fiction",
    "Sir Walter Raleigh and his Time": "History/Biography, Non-Fiction",
    "The Ancien Regime": "History/Lectures, Non-Fiction",
    "The Good News of God": "Religion/Sermons, Non-Fiction",
    "The Gospel of the Pentateuch": "Religion/Sermons, Non-Fiction",
    "The Hermits": "History/Religion, Non-Fiction",
    "The Heroes": "Mythology/Children's, Fiction",
    "The Roman and the Teuton": "History/Lectures, Non-Fiction",
    "The Saint's Tragedy": "Drama/Historical, Fiction",
    "The Water of Life and Other Sermons": "Religion/Sermons, Non-Fiction",
    "The Water-Babies": "Children's/Fantasy, Fiction",
    "Town and Country Sermons": "Religion/Sermons, Non-Fiction",
    "Town Geology": "Science/Geology, Non-Fiction",
    "True Words for Brave Men": "Religion/Inspirational, Non-Fiction",
    "Twenty-Five Village Sermons": "Religion/Sermons, Non-Fiction",
    "Two Years Ago, Volume I": "Literature/Novel, Fiction",
    "Two Years Ago, Volume II": "Literature/Novel, Fiction",
    "Westminster Sermons": "Religion/Sermons, Non-Fiction",
    "Westward Ho!": "Adventure/Historical, Fiction",
    "Women and Politics": "Politics/Social, Non-Fiction",
    "Yeast_ A Problem": "Literature/Novel, Fiction",
    "Jane Eyre": "Literature/Romance, Fiction",
    "Shirley": "Literature/Novel, Fiction",
    "The Professor": "Literature/Novel, Fiction",
    "Villette": "Literature/Novel, Fiction",
    "A Book of Golden Deeds": "History/Inspirational, Non-Fiction",
    "A Modern Telemachus": "Adventure/Novel, Fiction",
    "A Reputed Changeling": "Historical/Novel, Fiction",
    "Abbeychurch": "Literature/Novel, Fiction",
    "Aunt Charlotte's Stories of Greek History": "History/Children's, Non-Fiction",
    "Beechcroft at Rockstone": "Literature/Novel, Fiction",
    "Cameos from English History, from Rollo to Edward II": "History, Non-Fiction",
    "Chantry House": "Literature/Novel, Fiction",
    "Countess Kate": "Children's/Novel, Fiction",
    "Dynevor Terrace (Vol. I)": "Literature/Novel, Fiction",
    "Dynevor Terrace (Vol. II)": "Literature/Novel, Fiction",
    "Friarswood Post-Office": "Children's/Novel, Fiction",
    "Grisly Grisell": "Historical/Novel, Fiction",
    "Heartsease": "Literature/Novel, Fiction",
    "Henrietta's Wish": "Literature/Novel, Fiction",
    "History of France": "History, Non-Fiction",
    "Hopes and Fears": "Literature/Novel, Fiction",
    "John Keble's Parishes": "History/Religion, Non-Fiction",
    "Lady Hester, or Ursula's Narrative": "Literature/Novel, Fiction",
    "Life of John Coleridge Patteson": "Biography/Religion, Non-Fiction",
    "Little Lucy's Wonderful Globe": "Children's/Geography, Fiction",
    "Love and Life": "Literature/Novel, Fiction",
    "Magnum Bonum": "Literature/Novel, Fiction",
    "Modern Broods": "Literature/Novel, Fiction",
    "More Bywords": "Literature/Short Stories, Fiction",
    "My Young Alcides": "Literature/Novel, Fiction",
    "Nuttie's Father": "Literature/Novel, Fiction",
    "Old Times at Otterbourne": "History/Memoir, Non-Fiction",
    "Pioneers and Founders": "History/Biography, Non-Fiction",
    "Scenes and Characters": "Literature/Novel, Fiction",
    "Sowing and Sewing": "Literature/Novel, Fiction",
    "Stray Pearls": "Historical/Novel, Fiction",
    "That Stick": "Literature/Novel, Fiction",
    "The Armourer's Prentices": "Historical/Novel, Fiction",
    "The Caged Lion": "Historical/Novel, Fiction",
    "The Carbonels": "Literature/Novel, Fiction",
    "The Chaplet of Pearls": "Historical/Novel, Fiction",
    "The Chosen People": "Religion/History, Non-Fiction",
    "The Clever Woman of the Family": "Literature/Novel, Fiction",
    "The Daisy Chain": "Literature/Novel, Fiction",
    "The Dove in the Eagle's Nest": "Historical/Novel, Fiction",
    "The Heir of Redclyffe": "Literature/Novel, Fiction",
    "The Herd Boy and His Hermit": "Historical/Children's, Fiction",
    "The Lances of Lynwood": "Historical/Adventure, Fiction",
    "The Little Duke": "Historical/Children's, Fiction",
    "The Long Vacation": "Literature/Novel, Fiction",
    "The Pigeon Pie": "Historical/Novel, Fiction",
    "The Pillars of the House, V1": "Literature/Novel, Fiction",
    "The Prince and the Page": "Historical/Adventure, Fiction",
    "The Stokesley Secret": "Children's/Novel, Fiction",
    "The Three Brides": "Literature/Novel, Fiction",
    "The Trial": "Literature/Novel, Fiction",
    "The Two Guardians": "Literature/Novel, Fiction",
    "The Two Sides of the Shield": "Literature/Novel, Fiction",
    "The Young Step-Mother": "Literature/Novel, Fiction",
    "Two Penniless Princesses": "Historical/Novel, Fiction",
    "Under the Storm": "Historical/Novel, Fiction",
    "Unknown to History": "Historical/Novel, Fiction",
    "Young Folks' History of England": "History/Children's, Non-Fiction",
    "Young Folks' History of Rome": "History/Children's, Non-Fiction",
    "Aaron's Rod": "Literature/Novel, Fiction",
    "Amores": "Poetry, Fiction",
    "Bay": "Poetry, Fiction",
    "England, My England": "Literature/Short Stories, Fiction",
    "Fantasia of the Unconscious": "Philosophy/Psychology, Non-Fiction",
    "Look! We Have Come Through!": "Poetry, Fiction",
    "New Poems": "Poetry, Fiction",
    "Sea and Sardinia": "Travel, Non-Fiction",
    "Sons and Lovers": "Literature/Novel, Fiction",
    "The Lost Girl": "Literature/Novel, Fiction",
    "The Prussian Officer": "Literature/Short Stories, Fiction",
    "The Rainbow": "Literature/Novel, Fiction",
    "The Trespasser": "Literature/Novel, Fiction",
    "The White Peacock": "Literature/Novel, Fiction",
    "Tortoises": "Poetry, Fiction",
    "Touch and Go": "Drama, Fiction",
    "Twilight in Italy": "Travel, Non-Fiction",
    "Wintry Peacock": "Literature/Short Story, Fiction",
    "Women in Love": "Literature/Novel, Fiction",
    "A Journal of the Plague Year": "Historical/Fiction, Fiction",
    "A New Voyage Round the World by a Course Never Sailed Before": "Adventure/Fiction, Fiction",
    "A Seasonable Warning and Caution against the Insinuations of Papists and Jacobites": "Politics/Essay, Non-Fiction",
    "A Short Narrative of the Life and Actions of His Grace John, D. of Marlborogh": "Biography, Non-Fiction",
    "A True Relation of the Apparition of one Mrs. Veal": "Supernatural/Short Story, Fiction",
    "A Vindication of the Press": "Literature/Essay, Non-Fiction",
    "An Answer to a Question that Nobody thinks of, viz., But what if the Queen should Die_": "Politics/Essay, Non-Fiction",
    "An Appeal to Honour and Justice, Though It Be of His Worst Enemies": "Biography/Defense, Non-Fiction",
    "An Essay Upon Projects": "Economics/Essay, Non-Fiction",
    "An Humble Proposal to the People of England": "Politics/Essay, Non-Fiction",
    "And What if the Pretender should Come_": "Politics/Essay, Non-Fiction",
    "Atalantis Major": "Politics/Satire, Fiction",
    "Augusta Triumphans": "Social/Essay, Non-Fiction",
    "Dickory Cronke": "Philosophy/Fiction, Fiction",
    "Everybody's Business is Nobody's Business": "Social/Essay, Non-Fiction",
    "From London to Land's End": "Travel, Non-Fiction",
    "History of the Plague in London": "Historical, Non-Fiction",
    "Memoirs of a Cavalier": "Historical/Fiction, Fiction",
    "Memoirs of Major Alexander Ramkins (1718)": "Adventure/Fiction, Fiction",
    "Military Memoirs of Capt. George Carleton": "Biography/Military, Non-Fiction",
    "Of Captain Mission": "Adventure/Piracy, Fiction",
    "Reasons against the Succession of the House of Hanover": "Politics/Essay, Non-Fiction",
    "Second Thoughts are Best": "Social/Essay, Non-Fiction",
    "The Complete English Tradesman (1839 ed.)": "Economics/Business, Non-Fiction",
    "The Consolidator": "Science Fiction/Satire, Fiction",
    "The Fortunate Mistress (Parts 1 and 2)": "Literature/Novel, Fiction",
    "The Fortunes and Misfortunes of the Famous Moll Flanders": "Literature/Novel, Fiction",
    "The Further Adventures of Robinson Crusoe": "Adventure/Novel, Fiction",
    "The History of the Devil": "Religion/Satire, Non-Fiction",
    "The History of the Life and Adventures of Mr. Duncan Campbell": "Biography/Fiction, Fiction",
    "The History of the Remarkable Life of John Sheppard": "Biography/Crime, Non-Fiction",
    "The King of Pirates": "Adventure/Piracy, Fiction",
    "The Lay-Man's Sermon upon the Late Storm": "Religion/Sermon, Non-Fiction",
    "The Life and Adventures of Robinson Crusoe (1808)": "Adventure/Novel, Fiction",
    "The Life and Adventures of Robinson Crusoe Of York, Mariner, Vol. 1": "Adventure/Novel, Fiction",
    "The Life and Adventures of Robinson Crusoe": "Adventure/Novel, Fiction",
    "The Life and Most Surprising Adventures of Robinson Crusoe": "Adventure/Novel, Fiction",
    "The Life, Adventures & Piracies of the Famous Captain Singleton": "Adventure/Novel, Fiction",
    "The True-Born Englishman": "Poetry/Satire, Fiction",
    "Tour through the Eastern Counties of England, 1722": "Travel, Non-Fiction",
    "Edgar Allan Poe's Complete Poetical Works": "Poetry/Horror, Fiction",
    "Eureka": "Science/Philosophy, Non-Fiction",
    "The Cask of Amontillado": "Horror/Short Story, Fiction",
    "The Fall of the House of Usher": "Horror/Short Story, Fiction",
    "The Masque of the Red Death": "Horror/Short Story, Fiction",
    "The Raven": "Poetry/Horror, Fiction",
    "The Works of Edgar Allan Poe Volume 1": "Horror/Collection, Fiction",
    "The Works of Edgar Allan Poe Volume 2": "Horror/Collection, Fiction",
    "The Works of Edgar Allan Poe Volume 3": "Horror/Collection, Fiction",
    "The Works of Edgar Allan Poe Volume 4": "Horror/Collection, Fiction",
    "The Works of Edgar Allan Poe Volume 5": "Horror/Collection, Fiction",
    "A Princess of Mars": "Science Fiction/Adventure, Fiction",
    "At the Earth's Core": "Science Fiction/Adventure, Fiction",
    "Jungle Tales of Tarzan": "Adventure/Short Stories, Fiction",
    "Out of Time's Abyss": "Science Fiction/Adventure, Fiction",
    "Pellucidar": "Science Fiction/Adventure, Fiction",
    "Tarzan and the Jewels of Opar": "Adventure/Novel, Fiction",
    "Tarzan of the Apes": "Adventure/Novel, Fiction",
    "Tarzan the Terrible": "Adventure/Novel, Fiction",
    "Tarzan the Untamed": "Adventure/Novel, Fiction",
    "The Beasts of Tarzan": "Adventure/Novel, Fiction",
    "The Chessmen of Mars": "Science Fiction/Adventure, Fiction",
    "The Efficiency Expert": "Adventure/Novel, Fiction",
    "The Gods of Mars": "Science Fiction/Adventure, Fiction",
    "The Land That Time Forgot": "Science Fiction/Adventure, Fiction",
    "The Lost Continent": "Science Fiction/Adventure, Fiction",
    "The Mad King": "Adventure/Novel, Fiction",
    "The Monster Men": "Science Fiction/Adventure, Fiction",
    "The Mucker": "Adventure/Novel, Fiction",
    "The Oakdale Affair": "Adventure/Novel, Fiction",
    "The Outlaw of Torn": "Historical/Adventure, Fiction",
    "The People that Time Forgot": "Science Fiction/Adventure, Fiction",
    "The Return of Tarzan": "Adventure/Novel, Fiction",
    "The Son of Tarzan": "Adventure/Novel, Fiction",
    "Thuvia, Maid of Mars": "Science Fiction/Adventure, Fiction",
    "Warlord of Mars": "Science Fiction/Adventure, Fiction",
    "Burke's Speech on Conciliation with America": "Politics/Speech, Non-Fiction",
    "Selections from the Speeches and Writings of Edmund Burke": "Politics/Collection, Non-Fiction"
    "The Works of the Right Honourable Edmund Burke, Vol. 01 (of 12)": "Politics/Philosophy, Non-Fiction",
    "The Works of the Right Honourable Edmund Burke, Vol. 02 (of 12)": "Politics/Philosophy, Non-Fiction",
    "The Works of the Right Honourable Edmund Burke, Vol. 03 (of 12)": "Politics/Philosophy, Non-Fiction",
    "The Works of the Right Honourable Edmund Burke, Vol. 04 (of 12)": "Politics/Philosophy, Non-Fiction",
    "The Works of the Right Honourable Edmund Burke, Vol. 05 (of 12)": "Politics/Philosophy, Non-Fiction",
    "The Works of the Right Honourable Edmund Burke, Vol. 06 (of 12)": "Politics/Philosophy, Non-Fiction",
    "The Works of the Right Honourable Edmund Burke, Vol. 07 (of 12)": "Politics/Philosophy, Non-Fiction",
    "The Works of the Right Honourable Edmund Burke, Vol. 08 (of 12)": "Politics/Philosophy, Non-Fiction",
    "The Works of the Right Honourable Edmund Burke, Vol. 09 (of 12)": "Politics/Philosophy, Non-Fiction",
    "The Works of the Right Honourable Edmund Burke, Vol. 10 (of 12)": "Politics/Philosophy, Non-Fiction",
    "The Works of the Right Honourable Edmund Burke, Vol. 11 (of 12)": "Politics/Philosophy, Non-Fiction",
    "The Works of the Right Honourable Edmund Burke, Vol. 12 (of 12)": "Politics/Philosophy, Non-Fiction",
    "Thoughts on the Present Discontents": "Politics/Essay, Non-Fiction",
    "A Lost Leader": "Mystery/Thriller, Fiction",
    "A Maker of History": "Mystery/Adventure, Fiction",
    "A Millionaire of Yesterday": "Adventure/Romance, Fiction",
    "A Monk of Cruta": "Mystery/Adventure, Fiction",
    "A People's Man": "Mystery/Political, Fiction",
    "A Prince of Sinners": "Mystery/Adventure, Fiction",
    "An Amiable Charlatan": "Mystery/Adventure, Fiction",
    "Anna the Adventuress": "Mystery/Adventure, Fiction",
    "Berenice": "Mystery/Thriller, Fiction",
    "Havoc": "Mystery/Spy, Fiction",
    "Jacob's Ladder": "Mystery/Adventure, Fiction",
    "Jeanne of the Marshes": "Mystery/Romance, Fiction",
    "Mr. Grex of Monte Carlo": "Mystery/Spy, Fiction",
    "Mr. Marx's Secret": "Mystery/Adventure, Fiction",
    "Mysterious Mr. Sabin": "Mystery/Spy, Fiction",
    "Nobody's Man": "Mystery/Political, Fiction",
    "Peter Ruff and the Double Four": "Mystery/Detective, Fiction",
    "The Avenger": "Mystery/Thriller, Fiction",
    "The Betrayal": "Mystery/Spy, Fiction",
    "The Black Box": "Mystery/Adventure, Fiction",
    "The Box with the Broken Seals": "Mystery/Detective, Fiction",
    "The Cinema Murder": "Mystery/Crime, Fiction",
    "The Devil's Paw": "Mystery/Spy, Fiction",
    "The Double Four": "Mystery/Short Stories, Fiction",
    "The Double Life Of Mr. Alfred Burton": "Mystery/Comedy, Fiction",
    "The Double Traitor": "Mystery/Spy, Fiction",
    "The Evil Shepherd": "Mystery/Crime, Fiction",
    "The Golden Web": "Mystery/Adventure, Fiction",
    "The Governors": "Mystery/Political, Fiction",
    "The Great Impersonation": "Mystery/Spy, Fiction",
    "The Great Prince Shan": "Mystery/Spy, Fiction",
    "The Great Secret": "Mystery/Adventure, Fiction",
    "The Hillman": "Mystery/Romance, Fiction",
    "The Illustrious Prince": "Mystery/Spy, Fiction",
    "The Kingdom of the Blind": "Mystery/Spy, Fiction",
    "The Lighted Way": "Mystery/Adventure, Fiction",
    "The Lost Ambassador": "Mystery/Spy, Fiction",
    "The Malefactor": "Mystery/Crime, Fiction",
    "The Master Mummer": "Mystery/Romance, Fiction",
    "The Mischief Maker": "Mystery/Spy, Fiction",
    "The Missioner": "Mystery/Adventure, Fiction",
    "The Moving Finger": "Mystery/Crime, Fiction",
    "The New Tenant": "Mystery/Thriller, Fiction",
    "The Pawns Count": "Mystery/Spy, Fiction",
    "The Profiteers": "Mystery/Adventure, Fiction",
    "The Survivor": "Mystery/Adventure, Fiction",
    "The Tempting of Tavernake": "Mystery/Romance, Fiction",
    "The Traitors": "Mystery/Political, Fiction",
    "The Vanished Messenger": "Mystery/Spy, Fiction",
    "The Wicked Marquis": "Mystery/Romance, Fiction",
    "The Yellow Crayon": "Mystery/Spy, Fiction",
    "The Zeppelin's Passenger": "Mystery/Spy, Fiction",
    "To Win the Love He Sought": "Mystery/Romance, Fiction",
    "American Boy's Life of Theodore Roosevelt": "Biography/History, Non-Fiction",
    "At the Fall of Port Arthur": "Adventure/War, Fiction",
    "Boys of The Fort": "Adventure/Western, Fiction",
    "Dave Porter and His Double": "Young Adult/Adventure, Fiction",
    "Dave Porter and His Rivals": "Young Adult/Adventure, Fiction",
    "Dave Porter and the Runaways": "Young Adult/Adventure, Fiction",
    "Dave Porter At Bear Camp": "Young Adult/Adventure, Fiction",
    "Dave Porter at Star Ranch": "Young Adult/Adventure, Fiction",
    "Dave Porter in the Far North": "Young Adult/Adventure, Fiction",
    "Dave Porter in the Gold Fields": "Young Adult/Adventure, Fiction",
    "Dave Porter in the South Seas": "Young Adult/Adventure, Fiction",
    "Dave Porter on Cave Island": "Young Adult/Adventure, Fiction",
    "Fighting in Cuban Waters": "Adventure/War, Fiction",
    "First at the North Pole": "Adventure/Exploration, Fiction",
    "For the Liberty of Texas": "Historical/Adventure, Fiction",
    "Four Boy Hunters": "Young Adult/Adventure, Fiction",
    "Guns And Snowshoes": "Young Adult/Adventure, Fiction",
    "Leo the Circus Boy": "Young Adult/Adventure, Fiction",
    "Marching on Niagara": "Historical/Adventure, Fiction",
    "On the Trail of Pontiac": "Historical/Adventure, Fiction",
    "Out with Gun and Camera": "Young Adult/Adventure, Fiction",
    "Richard Dare's Venture": "Young Adult/Adventure, Fiction",
    "The Boy Land Boomer": "Western/Adventure, Fiction",
    "The Campaign of the Jungle": "Adventure/War, Fiction",
    "The Last Cruise of the Spitfire": "Adventure/Sea, Fiction",
    "The Missing Tin Box": "Mystery/Adventure, Fiction",
    "The Mystery at Putnam Hall": "Young Adult/Mystery, Fiction",
    "The Rover Boys at Big Horn Ranch": "Young Adult/Adventure, Fiction",
    "The Rover Boys at Colby Hall": "Young Adult/Adventure, Fiction",
    "The Rover Boys at College": "Young Adult/Adventure, Fiction",
    "The Rover Boys at School": "Young Adult/Adventure, Fiction",
    "The Rover Boys in Alaska": "Young Adult/Adventure, Fiction",
    "The Rover Boys in Business": "Young Adult/Adventure, Fiction",
    "The Rover Boys in Camp": "Young Adult/Adventure, Fiction",
    "The Rover Boys in New York": "Young Adult/Adventure, Fiction",
    "The Rover Boys in Southern Waters": "Young Adult/Adventure, Fiction",
    "The Rover Boys in the Air": "Young Adult/Adventure, Fiction",
    "The Rover Boys in the Jungle": "Young Adult/Adventure, Fiction",
    "The Rover Boys in the Land of Luck": "Young Adult/Adventure, Fiction",
    "The Rover Boys In The Mountains": "Young Adult/Adventure, Fiction",
    "The Rover Boys on a Hunt": "Young Adult/Adventure, Fiction",
    "The Rover Boys on a Tour": "Young Adult/Adventure, Fiction",
    "The Rover Boys on Land and Sea": "Young Adult/Adventure, Fiction",
    "The Rover Boys on Snowshoe Island": "Young Adult/Adventure, Fiction",
    "The Rover Boys on the Farm": "Young Adult/Adventure, Fiction",
    "The Rover Boys on the Great Lakes": "Young Adult/Adventure, Fiction",
    "The Rover Boys on the Ocean": "Young Adult/Adventure, Fiction",
    "The Rover Boys on the Plains": "Young Adult/Adventure, Fiction",
    "The Rover Boys on the River": "Young Adult/Adventure, Fiction",
    "The Rover Boys on Treasure Isle": "Young Adult/Adventure, Fiction",
    "The Rover Boys out West": "Young Adult/Adventure, Fiction",
    "The Young Bridge-Tender": "Young Adult/Adventure, Fiction",
    "The Young Oarsmen of Lakeview": "Young Adult/Adventure, Fiction",
    "Three Young Ranchmen": "Young Adult/Western, Fiction",
    "To Alaska for Gold": "Adventure/Exploration, Fiction",
    "True to Himself": "Young Adult/Adventure, Fiction",
    "Young Auctioneers": "Young Adult/Adventure, Fiction",
    "Young Hunters of the Lake": "Young Adult/Adventure, Fiction",
    "'He Giveth His Beloved Sleep'": "Poetry, Fiction",
    "Sonnets from the Portuguese": "Poetry, Fiction",
    "The Letters of Elizabeth Barrett Browning (1 of 2)": "Biography/Letters, Non-Fiction",
    "The Letters of Elizabeth Barrett Browning (2 of 2)": "Biography/Letters, Non-Fiction",
    "The Poetical Works of Elizabeth Barrett Browning Volume I": "Poetry, Fiction",
    "The Poetical Works of Elizabeth Barrett Browning Volume II": "Poetry, Fiction",
    "The Poetical Works of Elizabeth Barrett Browning Volume IV": "Poetry, Fiction",
    "Wuthering Heights": "Literature/Gothic, Fiction",
    "Anna Christie": "Drama, Fiction",
    "The First Man": "Drama, Fiction",
    "The Hairy Ape": "Drama, Fiction",
    "The Straw": "Drama, Fiction",
    "Certain Noble Plays of Japan": "Drama/Literature, Non-Fiction",
    "Hugh Selwyn Mauberley": "Poetry, Fiction",
    "A Bicycle of Cathay": "Literature/Romance, Fiction",
    "A Chosen Few": "Literature/Short Stories, Fiction",
    "A Jolly Fellowship": "Young Adult/Adventure, Fiction",
    "Amos Kilbright; His Adscititious Experiences": "Fantasy/Short Stories, Fiction",
    "Buccaneers and Pirates of Our Coasts": "History/Adventure, Non-Fiction",
    "John Gayther's Garden and the Stories Told Therein": "Literature/Short Stories, Fiction",
    "Kate Bonnet": "Adventure/Pirate, Fiction",
    "Mrs. Cliff's Yacht": "Adventure/Humor, Fiction",
    "My Terminal Moraine": "Adventure/Short Story, Fiction",
    "Pomona's Travels": "Travel/Humor, Fiction",
    "Round-about Rambles in Lands of Fact and Fancy": "Children's/Fantasy, Fiction",
    "Rudder Grange": "Literature/Humor, Fiction",
    "Stories of New Jersey": "History/Short Stories, Fiction",
    "The Adventures of Captain Horn": "Adventure/Novel, Fiction",
    "The Associate Hermits": "Literature/Humor, Fiction",
    "The Bee-Man of Orn and Other Fanciful Tales": "Fantasy/Short Stories, Fiction",
    "The Captain's Toll-Gate": "Literature/Novel, Fiction",
    "The Casting Away of Mrs. Lecks and Mrs. Aleshine": "Adventure/Humor, Fiction",
    "The Dusantes": "Literature/Short Story, Fiction",
    "The Girl at Cobhurst": "Literature/Romance, Fiction",
    "The Great Stone of Sardis": "Science Fiction/Adventure, Fiction",
    "The Great War Syndicate": "Science Fiction/War, Fiction",
    "The House of Martha": "Literature/Romance, Fiction",
    "The Lady, or the Tiger_": "Literature/Short Story, Fiction",
    "The Late Mrs. Null": "Literature/Novel, Fiction",
    "The Magic Egg and Other Stories": "Fantasy/Short Stories, Fiction",
    "The Rudder Grangers Abroad and Other Stories": "Literature/Short Stories, Fiction",
    "The Squirrel Inn": "Literature/Humor, Fiction",
    "The Stories of the Three Burglars": "Mystery/Short Stories, Fiction",
    "The Vizier of the Two-Horned Alexander": "Fantasy/Adventure, Fiction",
    "Ting-a-ling": "Fantasy/Children's, Fiction",
    "What Might Have Been Expected": "Young Adult/Adventure, Fiction",
    "A Miscellany of Men": "Literature/Essays, Non-Fiction",
    "A Short History of England": "History, Non-Fiction",
    "Alarms and Discursions": "Literature/Essays, Non-Fiction",
    "All Things Considered": "Literature/Essays, Non-Fiction",
    "Appreciations and Criticisms of the Works of Charles Dickens": "Literature/Criticism, Non-Fiction",
    "Eugenics and Other Evils": "Philosophy/Social, Non-Fiction",
    "George Bernard Shaw": "Biography/Literature, Non-Fiction",
    "Greybeards at Play": "Poetry/Humor, Fiction",
    "Heretics": "Philosophy/Religion, Non-Fiction",
    "Lord Kitchener": "Biography/Military, Non-Fiction",
    "Magic": "Drama/Fantasy, Fiction",
    "Manalive": "Literature/Mystery, Fiction",
    "Orthodoxy": "Philosophy/Religion, Non-Fiction",
    "Poems": "Poetry, Fiction",
    "Robert Browning": "Biography/Literature, Non-Fiction",
    "The Appetite of Tyranny": "Politics/Essay, Non-Fiction",
    "The Ball and The Cross": "Literature/Adventure, Fiction",
    "The Ballad of St. Barbara": "Poetry, Fiction",
    "The Ballad of the White Horse": "Poetry/Epic, Fiction",
    "The Barbarism of Berlin": "Politics/Essay, Non-Fiction",
    "The Club of Queer Trades": "Mystery/Humor, Fiction",
    "The Crimes of England": "History/Politics, Non-Fiction",
    "The Defendant": "Literature/Essays, Non-Fiction",
    "The Innocence of Father Brown": "Mystery/Detective, Fiction",
    "The Man Who Knew Too Much": "Mystery/Detective, Fiction",
    "The Man Who Was Thursday": "Mystery/Thriller, Fiction",
    "The Napoleon of Notting Hill": "Science Fiction/Satire, Fiction",
    "The New Jerusalem": "Travel/History, Non-Fiction",
    "The Trees of Pride": "Mystery/Novella, Fiction",
    "The Victorian Age in Literature": "Literature/Criticism, Non-Fiction",
    "The Wild Knight and Other Poems": "Poetry, Fiction",
    "The Wisdom of Father Brown": "Mystery/Detective, Fiction",
    "Tremendous Trifles": "Literature/Essays, Non-Fiction",
    "Twelve Types": "Literature/Essays, Non-Fiction",
    "Utopia of Usurers and other Essays": "Politics/Essays, Non-Fiction",
    "Varied Types": "Literature/Essays, Non-Fiction",
    "What I Saw in America": "Travel/Social, Non-Fiction",
    "What's Wrong With The World": "Philosophy/Social, Non-Fiction",
    "Wine, Water, and Song": "Poetry/Humor, Fiction",
    "A Chapter of Adventures": "Young Adult/Adventure, Fiction",
    "A Final Reckoning": "Historical/Adventure, Fiction",
    "A Girl of the Commune": "Historical/Adventure, Fiction",
    "A Jacobite Exile": "Historical/Adventure, Fiction",
    "A Knight of the White Cross": "Historical/Adventure, Fiction",
    "A March on London": "Historical/Adventure, Fiction",
    "A Roving Commission": "Historical/Adventure, Fiction",
    "A Search For A Secret, a Novel, Volume 1": "Mystery/Novel, Fiction",
    "A Search For A Secret, a Novel, Volume 2": "Mystery/Novel, Fiction",
    "A Search For A Secret, a Novel, Volume 3": "Mystery/Novel, Fiction",
    "Among Malay Pirates": "Adventure/Sea, Fiction",
    "At Aboukir and Acre": "Historical/Adventure, Fiction",
    "At Agincourt": "Historical/Adventure, Fiction",
    "At the Point of the Bayonet": "Historical/Adventure, Fiction",
    "Beric the Briton": "Historical/Adventure, Fiction",
    "Bonnie Prince Charlie": "Historical/Adventure, Fiction",
    "Both Sides the Border": "Historical/Adventure, Fiction",
    "By Conduct and Courage": "Historical/Adventure, Fiction",
    "By England's Aid or The Freeing of the Netherlands (1585-1604)": "Historical/Adventure, Fiction",
    "By Pike and Dyke_  A Tale of the Rise of the Dutch Republic": "Historical/Adventure, Fiction",
    "By Right of Conquest": "Historical/Adventure, Fiction",
    "By Sheer Pluck": "Historical/Adventure, Fiction",
    "Captain Bayley's Heir": "Adventure/Western, Fiction",
    "Colonel Thorndyke's Secret": "Mystery/Adventure, Fiction",
    "Condemned as a Nihilist": "Historical/Adventure, Fiction",
    "Dorothy's Double, Volume 1": "Mystery/Novel, Fiction",
    "Dorothy's Double, Volume 2": "Mystery/Novel, Fiction",
    "Dorothy's Double, Volume 3": "Mystery/Novel, Fiction",
    "Facing Death": "Adventure/Novel, Fiction",
    "For Name and Fame": "Historical/Adventure, Fiction",
    "For the Temple": "Historical/Adventure, Fiction",
    "Friends, though divided": "Historical/Adventure, Fiction",
    "Held Fast For England": "Historical/Adventure, Fiction",
    "In Freedom's Cause": "Historical/Adventure, Fiction",
    "In the Hands of the Malays, and Other Stories": "Adventure/Short Stories, Fiction",
    "In The Heart Of The Rockies": "Adventure/Western, Fiction",
    "In the Irish Brigade": "Historical/Adventure, Fiction",
    "In the Reign of Terror": "Historical/Adventure, Fiction",
    "In Times of Peril": "Historical/Adventure, Fiction",
    "Jack Archer": "Historical/Adventure, Fiction",
    "Maori and Settler": "Historical/Adventure, Fiction",
    "March to Magdala": "Historical/Adventure, Fiction",
    "No Surrender!": "Historical/Adventure, Fiction",
    "On the Irrawaddy": "Historical/Adventure, Fiction",
    "On the Pampas": "Adventure/Western, Fiction",
    "One of the 28th": "Historical/Adventure, Fiction",
    "Orange and Green": "Historical/Adventure, Fiction",
    "Out on the Pampas": "Adventure/Western, Fiction",
    "Rujub, the Juggler": "Historical/Adventure, Fiction",
    "Saint Bartholomew's Eve": "Historical/Adventure, Fiction",
    "Saint George for England": "Historical/Adventure, Fiction",
    "Sturdy and Strong": "Young Adult/Adventure, Fiction",
    "Tales of Daring and Danger": "Adventure/Short Stories, Fiction",
    "The Boy Knight": "Historical/Adventure, Fiction",
    "The Bravest of the Brave": "Historical/Adventure, Fiction",
    "The Cat of Bubastes": "Historical/Adventure, Fiction",
    "The Cornet of Horse": "Historical/Adventure, Fiction",
    "The Curse of Carne's Hold": "Mystery/Adventure, Fiction",
    "The Dash for Khartoum": "Historical/Adventure, Fiction",
    "The Dragon and the Raven": "Historical/Adventure, Fiction",
    "The Golden Canyon": "Adventure/Western, Fiction",
    "The Lion of Saint Mark": "Historical/Adventure, Fiction",
    "The Lion of the North": "Historical/Adventure, Fiction",
    "The Lost Heir": "Mystery/Adventure, Fiction",
    "The Queen's Cup": "Adventure/Romance, Fiction",
    "The Tiger of Mysore": "Historical/Adventure, Fiction",
    "The Treasure of the Incas": "Adventure/Exploration, Fiction",
    "The Young Buglers": "Historical/Adventure, Fiction",
    "The Young Carthaginian": "Historical/Adventure, Fiction",
    "The Young Colonists": "Historical/Adventure, Fiction",
    "The Young Franc Tireurs": "Historical/Adventure, Fiction",
    "Through Russian Snows": "Historical/Adventure, Fiction",
    "Through the Fray": "Historical/Adventure, Fiction",
    "Through Three Campaigns": "Historical/Adventure, Fiction",
    "True to the Old Flag": "Historical/Adventure, Fiction",
    "Under Drake's Flag": "Historical/Adventure, Fiction",
    "Under Wellington's Command": "Historical/Adventure, Fiction",
    "When London Burned": "Historical/Adventure, Fiction",
    "Winning His Spurs": "Historical/Adventure, Fiction",
    "With Buller in Natal": "Historical/Adventure, Fiction",
    "With Clive in India": "Historical/Adventure, Fiction",
    "With Cochrane the Dauntless": "Historical/Adventure, Fiction",
    "With Frederick the Great": "Historical/Adventure, Fiction",
    "With Kitchener in the Soudan": "Historical/Adventure, Fiction",
    "With Lee in Virginia": "Historical/Adventure, Fiction",
    "With Moore At Corunna": "Historical/Adventure, Fiction",
    "With Wolfe in Canada": "Historical/Adventure, Fiction",
    "Won by the Sword": "Historical/Adventure, Fiction",
    "Wulf the Saxon": "Historical/Adventure, Fiction",
    "A Treatise on Parents and Children": "Philosophy/Education, Non-Fiction",
    "An Unsocial Socialist": "Literature/Satire, Fiction",
    "Androcles and the Lion": "Drama/Comedy, Fiction",
    "Annajanska, the Bolshevik Empress": "Drama/Comedy, Fiction",
    "Arms and the Man": "Drama/Comedy, Fiction",
    "Augustus Does His Bit": "Drama/Comedy, Fiction",
    "Back to Methuselah": "Drama/Science Fiction, Fiction",
    "Bernard Shaw's Preface to Major Barbara": "Literature/Essay, Non-Fiction",
    "Caesar and Cleopatra": "Drama/Historical, Fiction",
    "Candida": "Drama, Fiction",
    "Captain Brassbound's Conversion": "Drama/Adventure, Fiction",
    "Cashel Byron's Profession": "Literature/Novel, Fiction",
    "Dark Lady of the Sonnets": "Drama/Comedy, Fiction",
    "Fanny's First Play": "Drama/Comedy, Fiction",
    "Getting Married": "Drama/Comedy, Fiction",
    "Great Catherine": "Drama/Historical, Fiction",
    "Heartbreak House": "Drama/Comedy, Fiction",
    "How He Lied to Her Husband": "Drama/Comedy, Fiction",
    "John Bull's Other Island": "Drama/Comedy, Fiction",
    "Major Barbara": "Drama/Comedy, Fiction",
    "Man And Superman": "Drama/Comedy, Fiction",
    "Maxims for Revolutionists": "Philosophy/Aphorisms, Non-Fiction",
    "Misalliance": "Drama/Comedy, Fiction",
    "Mrs. Warren's Profession": "Drama/Social, Fiction",
    "O'Flaherty V. C": "Drama/Comedy, Fiction",
    "Overruled": "Drama/Comedy, Fiction",
    "Preface to Androcles and the Lion": "Literature/Essay, Non-Fiction",
    "Press Cuttings": "Drama/Comedy, Fiction",
    "Pygmalion": "Drama/Comedy, Fiction",
    "Revolutionist's Handbook and Pocket Companion": "Philosophy/Satire, Non-Fiction",
    "The Admirable Bashville": "Drama/Comedy, Fiction",
    "The Devil's Disciple": "Drama/Historical, Fiction",
    "The Doctor's Dilemma": "Drama/Comedy, Fiction",
    "The Doctor's Dilemma_ Preface on Doctors": "Literature/Essay, Non-Fiction",
    "The Inca of Perusalem": "Drama/Comedy, Fiction",
    "The Irrational Knot": "Literature/Novel, Fiction",
    "The Man of Destiny": "Drama/Historical, Fiction",
    "The Miraculous Revenge": "Literature/Short Story, Fiction",
    "The Perfect Wagnerite": "Music/Criticism, Non-Fiction",
    "The Philanderer": "Drama/Comedy, Fiction",
    "The Shewing-up of Blanco Posnet": "Drama, Fiction",
    "You Never Can Tell": "Drama/Comedy, Fiction",
    "Adam Bede": "Literature/Novel, Fiction",
    "Brother Jacob": "Literature/Short Story, Fiction",
    "Daniel Deronda": "Literature/Novel, Fiction",
    "How Lisa Loved the King": "Poetry, Fiction",
    "Impressions of Theophrastus Such": "Literature/Essays, Fiction",
    "Middlemarch": "Literature/Novel, Fiction",
    "Romola": "Historical/Novel, Fiction",
    "Scenes of Clerical Life": "Literature/Short Stories, Fiction",
    "Silas Marner": "Literature/Novel, Fiction",
    "The Essays of George Eliot": "Literature/Essays, Non-Fiction",
    "The Lifted Veil": "Literature/Short Story, Fiction",
    "The Mill on the Floss": "Literature/Novel, Fiction",
    "Tom and Maggie Tulliver": "Children's/Literature, Fiction",
    "State of the Union Addresses of George Washington": "History/Politics, Non-Fiction",
    "An African Millionaire": "Mystery/Adventure, Fiction",
    "Anglo-Saxon Britain": "History, Non-Fiction",
    "Biographies of Working Men": "Biography, Non-Fiction",
    "Charles Darwin": "Biography/Science, Non-Fiction",
    "Falling in Love": "Science/Essays, Non-Fiction",
    "Hilda Wade": "Mystery/Adventure, Fiction",
    "Michael's Crag": "Literature/Novel, Fiction",
    "Miss Cayley's Adventures": "Mystery/Adventure, Fiction",
    "Philistia": "Literature/Novel, Fiction",
    "Post-Prandial Philosophy": "Philosophy/Essays, Non-Fiction",
    "Recalled to Life": "Science Fiction, Fiction",
    "Science in Arcady": "Science/Essays, Non-Fiction",
    "Strange Stories": "Fantasy/Short Stories, Fiction",
    "The Beckoning Hand and Other Stories": "Fantasy/Short Stories, Fiction",
    "The British Barbarians": "Science Fiction/Satire, Fiction",
    "The Great Taboo": "Adventure/Novel, Fiction",
    "The White Man's Foot": "Adventure/Novel, Fiction",
    "The Woman Who Did": "Literature/Novel, Fiction",
    "What's Bred In the Bone": "Mystery/Novel, Fiction",
    "A Daughter of the Middle Border": "Biography/Memoir, Non-Fiction",
    "A Little Norsk; Or, Ol' Pap's Flaxen": "Literature/Novel, Fiction",
    "A Son of the Middle Border": "Biography/Memoir, Non-Fiction",
    "A Spoil of Office": "Literature/Novel, Fiction",
    "Cavanaugh_ Forest Ranger": "Western/Adventure, Fiction",
    "Main-Travelled Roads": "Literature/Short Stories, Fiction",
    "Money Magic": "Literature/Novel, Fiction",
    "Other Main-Travelled Roads": "Literature/Short Stories, Fiction",
    "Prairie Folks": "Literature/Short Stories, Fiction",
    "Rose of Dutcher's Coolly": "Literature/Novel, Fiction",
    "The Captain of the Gray-Horse Troop": "Western/Adventure, Fiction",
    "The Eagle's Heart": "Western/Adventure, Fiction",
    "The Forester's Daughter": "Western/Romance, Fiction",
    "The Light of the Star": "Literature/Novel, Fiction",
    "The Moccasin Ranch": "Western/Novella, Fiction",
    "The Shadow World": "Supernatural/Fiction, Fiction",
    "The Spirit of Sweetwater": "Western/Romance, Fiction",
    "The Trail of the Goldseekers": "Adventure/Poetry, Fiction",
    "The Tyranny of the Dark": "Supernatural/Mystery, Fiction",
    "They of the High Trails": "Western/Short Stories, Fiction",
    "Victor Ollnee's Discipline": "Supernatural/Fiction, Fiction",
    "Wayside Courtships": "Literature/Short Stories, Fiction",
    "A Damaged Reputation": "Adventure/Western, Fiction",
    "A Prairie Courtship": "Western/Romance, Fiction",
    "Alton of Somasco": "Adventure/Western, Fiction",
    "Blake's Burden": "Adventure/Novel, Fiction",
    "Brandon of the Engineers": "Adventure/Novel, Fiction",
    "By Right of Purchase": "Western/Romance, Fiction",
    "Carmen's Messenger": "Mystery/Adventure, Fiction",
    "Delilah of the Snows": "Adventure/Romance, Fiction",
    "For Jacinta": "Adventure/Romance, Fiction",
    "For the Allinson Honor": "Adventure/Novel, Fiction",
    "Harding of Allenwood": "Western/Adventure, Fiction",
    "Hawtrey's Deputy": "Western/Adventure, Fiction",
    "Johnstone of the Border": "Adventure/Spy, Fiction",
    "Kit Musgrave's Luck": "Adventure/Sea, Fiction",
    "Lister's Great Adventure": "Adventure/Novel, Fiction",
    "Long Odds": "Adventure/Africa, Fiction",
    "Lorimer of the Northwest": "Western/Adventure, Fiction",
    "Masters of the Wheat-Lands": "Western/Adventure, Fiction",
    "Northwest!": "Western/Adventure, Fiction",
    "Partners of the Out-Trail": "Adventure/Western, Fiction",
    "Prescott of Saskatchewan": "Western/Mystery, Fiction",
    "Ranching for Sylvia": "Western/Romance, Fiction",
    "The Boy Ranchers of Puget Sound": "Young Adult/Adventure, Fiction",
    "The Buccaneer Farmer": "Adventure/Sea, Fiction",
    "The Cattle-Baron's Daughter": "Western/Romance, Fiction",
    "The Coast of Adventure": "Adventure/Novel, Fiction",
    "The Dust of Conflict": "Adventure/War, Fiction",
    "The Girl From Keller's": "Western/Romance, Fiction",
    "The Gold Trail": "Adventure/Western, Fiction",
    "The Greater Power": "Adventure/Novel, Fiction",
    "The Impostor": "Adventure/Novel, Fiction",
    "The Intriguers": "Adventure/Novel, Fiction",
    "The League of the Leopard": "Adventure/Africa, Fiction",
    "The Long Portage": "Adventure/Novel, Fiction",
    "The Lure of the North": "Adventure/Western, Fiction",
    "The Mistress of Bonaventure": "Western/Romance, Fiction",
    "The Protector": "Adventure/Sea, Fiction",
    "The Secret of the Reef": "Adventure/Sea, Fiction",
    "Thrice Armed": "Adventure/Sea, Fiction",
    "Thurston of Orchard Valley": "Western/Adventure, Fiction",
    "Vane of the Timberlands": "Adventure/Western, Fiction",
    "Winston of the Prairie": "Western/Adventure, Fiction",
    "Wyndham's Pal": "Adventure/Sea, Fiction",
    "Betty's Bright Idea": "Literature/Short Story, Fiction",
    "Household Papers and Stories": "Literature/Essays, Non-Fiction",
    "Lady Byron Vindicated": "Biography/Defense, Non-Fiction",
    "Oldtown Fireside Stories": "Literature/Short Stories, Fiction",
    "Pink and White Tyranny": "Literature/Novel, Fiction",
    "Queer Little Folks": "Children's/Stories, Fiction",
    "Sunny Memories Of Foreign Lands, Volume 1 (of 2)": "Travel/Memoir, Non-Fiction",
    "Sunny Memories Of Foreign Lands, Volume 2 (of 2)": "Travel/Memoir, Non-Fiction",
    "The May Flower, and Miscellaneous Writings": "Literature/Collection, Fiction",
    "The Pearl of Orr's Island": "Literature/Novel, Fiction",
    "Uncle Tom's Cabin, Young Folks' Edition": "Literature/Abolitionist, Fiction",
    "Uncle Tom's Cabin": "Literature/Abolitionist, Fiction",
    "Beasts and Super-Beasts": "Humor/Short Stories, Fiction",
    "Reginald in Russia": "Humor/Short Stories, Fiction",
    "Reginald": "Humor/Short Stories, Fiction",
    "The Chronicles of Clovis": "Humor/Short Stories, Fiction",
    "The Toys of Peace": "Humor/Short Stories, Fiction",
    "The Unbearable Bassington": "Literature/Satire, Fiction",
    "When William Came": "Science Fiction/Satire, Fiction",
    "A Plea for Captain John Brown": "History/Essay, Non-Fiction",
    "A Week on the Concord and Merrimack Rivers": "Travel/Philosophy, Non-Fiction",
    "Canoeing in the wilderness": "Travel/Adventure, Non-Fiction",
    "Cape Cod": "Travel/Nature, Non-Fiction",
    "Excursions": "Nature/Essays, Non-Fiction",
    "On the Duty of Civil Disobedience": "Philosophy/Politics, Non-Fiction",
    "Walden, and On The Duty Of Civil Disobedience": "Philosophy/Nature, Non-Fiction",
    "Walking": "Philosophy/Nature, Non-Fiction",
    "Wild Apples": "Nature/Essay, Non-Fiction",
    "Lives of the English Poets": "Biography/Literature, Non-Fiction",
    "A  Passionate Pilgrim": "Literature/Short Story, Fiction",
    "A Bundle of Letters": "Literature/Short Stories, Fiction",
    "A Little Tour in France, Version 1": "Travel, Non-Fiction",
    "A Little Tour in France, Version 2": "Travel, Non-Fiction",
    "A London Life and Other Tales": "Literature/Short Stories, Fiction",
    "A Small Boy and Others": "Biography/Memoir, Non-Fiction",
    "An International Episode": "Literature/Novella, Fiction",
    "Confidence": "Literature/Novel, Fiction",
    "Daisy Miller": "Literature/Novella, Fiction",
    "Embarrassments": "Literature/Short Stories, Fiction",
    "Eugene Pickering": "Literature/Short Story, Fiction",
    "Four Meetings": "Literature/Short Story, Fiction",
    "Georgina's Reasons": "Literature/Short Story, Fiction",
    "Glasses": "Literature/Short Story, Fiction",
    "Greville Fane": "Literature/Short Story, Fiction",
    "Hawthorne": "Biography/Literature, Non-Fiction",
    "In the Cage": "Literature/Novella, Fiction",
    "Italian Hours": "Travel/Essays, Non-Fiction",
    "Lady Barbarina": "Literature/Short Story, Fiction",
    "Louisa Pallant": "Literature/Short Story, Fiction",
    "Madame de Mauves": "Literature/Novella, Fiction",
    "Nona Vincent": "Literature/Short Story, Fiction",
    "Notes of a Son and Brother": "Biography/Memoir, Non-Fiction",
    "Pandora": "Literature/Short Story, Fiction",
    "Picture and Text": "Art/Essays, Non-Fiction",
    "Roderick Hudson": "Literature/Novel, Fiction",
    "Sir Dominick Ferrand": "Literature/Short Story, Fiction",
    "Some Short Stories": "Literature/Short Stories, Fiction",
    "The Altar of the Dead": "Literature/Short Story, Fiction",
    "The Ambassadors": "Literature/Novel, Fiction",
    "The American": "Literature/Novel, Fiction",
    "The Aspern Papers": "Literature/Novella, Fiction",
    "The Author Of Beltraffio": "Literature/Short Story, Fiction",
    "The Awkward Age": "Literature/Novel, Fiction",
    "The Beast in the Jungle": "Literature/Novella, Fiction",
    "The Beldonald Holbein": "Literature/Short Story, Fiction",
    "The Bostonians, Volume 1": "Literature/Novel, Fiction",
    "The Bostonians, Volume 2": "Literature/Novel, Fiction",
    "The Chaperon": "Literature/Short Story, Fiction",
    "The Coxon Fund": "Literature/Novella, Fiction",
    "The Death of the Lion": "Literature/Short Story, Fiction",
    "The Diary of a Man of Fifty": "Literature/Short Story, Fiction",
    "The Europeans": "Literature/Novel, Fiction",
    "The Figure in the Carpet": "Literature/Short Story, Fiction",
    "The Finer Grain": "Literature/Short Stories, Fiction",
    "The Golden Bowl": "Literature/Novel, Fiction",
    "The Jolly Corner": "Literature/Short Story, Fiction",
    "The Lesson of the Master": "Literature/Novella, Fiction",
    "The Letters of Henry James, Volume 1": "Biography/Letters, Non-Fiction",
    "The Letters of Henry James, Volume 2": "Biography/Letters, Non-Fiction",
    "The Madonna of the Future": "Literature/Short Story, Fiction",
    "The Marriages": "Literature/Short Story, Fiction",
    "The Middle Years": "Biography/Memoir, Non-Fiction",
    "The Outcry": "Literature/Novel, Fiction",
    "The Patagonia": "Literature/Short Story, Fiction",
    "The Path Of Duty": "Literature/Short Story, Fiction",
    "The Pension Beaurepas": "Literature/Short Story, Fiction",
    "The Point of View": "Literature/Short Story, Fiction",
    "The Portrait of a Lady, Volume 1": "Literature/Novel, Fiction",
    "The Portrait of a Lady, Volume 2": "Literature/Novel, Fiction",
    "The Pupil": "Literature/Short Story, Fiction",
    "The Real Thing": "Literature/Short Story, Fiction",
    "The Reverberator": "Literature/Novel, Fiction",
    "The Sacred Fount": "Literature/Novel, Fiction",
    "The Spoils of Poynton": "Literature/Novel, Fiction",
    "The Tragic Muse": "Literature/Novel, Fiction",
    "The Turn of the Screw": "Horror/Novella, Fiction",
    "The Wings of the Dove, Volume 1": "Literature/Novel, Fiction",
    "The Wings of the Dove, Volume 2": "Literature/Novel, Fiction",
    "Washington Square": "Literature/Novel, Fiction",
    "What Maisie Knew": "Literature/Novel, Fiction",
    "Within the Rim and Other Essays": "Literature/Essays, Non-Fiction",
    "A Tale of Three Lions": "Adventure/Short Story, Fiction",
    "Allan and the Holy Flower": "Adventure/Fantasy, Fiction",
    "Allan Quatermain": "Adventure/Novel, Fiction",
    "Allan's Wife": "Adventure/Short Stories, Fiction",
    "Ayesha": "Adventure/Fantasy, Fiction",
    "Beatrice": "Literature/Novel, Fiction",
    "Benita, An African Romance": "Adventure/Romance, Fiction",
    "Black Heart and White Heart": "Adventure/Short Stories, Fiction",
    "Cetywayo and his White Neighbours": "History/Politics, Non-Fiction",
    "Child of Storm": "Adventure/Novel, Fiction",
    "Cleopatra": "Historical/Adventure, Fiction",
    "Colonel Quaritch, V.C": "Adventure/Novel, Fiction",
    "Dawn": "Literature/Novel, Fiction",
    "Doctor Therne": "Literature/Novel, Fiction",
    "Elissa": "Historical/Adventure, Fiction",
    "Eric Brighteyes": "Historical/Adventure, Fiction",
    "Fair Margaret": "Historical/Adventure, Fiction",
    "Finished": "Adventure/Novel, Fiction",
    "Hunter Quatermain's Story": "Adventure/Short Story, Fiction",
    "Jess": "Adventure/Novel, Fiction",
    "King Solomon's Mines": "Adventure/Novel, Fiction",
    "Love Eternal": "Fantasy/Romance, Fiction",
    "Lysbeth": "Historical/Adventure, Fiction",
    "Maiwa's Revenge": "Adventure/Novella, Fiction",
    "Marie": "Adventure/Novel, Fiction",
    "Montezuma's Daughter": "Historical/Adventure, Fiction",
    "Moon of Israel": "Historical/Adventure, Fiction",
    "Morning Star": "Historical/Fantasy, Fiction",
    "Mr. Meeson's Will": "Adventure/Novel, Fiction",
    "Nada the Lily": "Adventure/Novel, Fiction",
    "Pearl-Maiden": "Historical/Adventure, Fiction",
    "Queen Sheba's Ring": "Adventure/Fantasy, Fiction",
    "Red Eve": "Historical/Adventure, Fiction",
    "Regeneration": "Social/Essay, Non-Fiction",
    "She and Allan": "Adventure/Fantasy, Fiction",
    "She": "Adventure/Fantasy, Fiction",
    "Smith and the Pharaohs, and Other Tales": "Adventure/Short Stories, Fiction",
    "Stella Fregelius": "Fantasy/Romance, Fiction",
    "Swallow": "Historical/Adventure, Fiction",
    "The Ancient Allan": "Adventure/Fantasy, Fiction",
    "The Brethren": "Historical/Adventure, Fiction",
    "The Ghost Kings": "Fantasy/Adventure, Fiction",
    "The Ivory Child": "Adventure/Fantasy, Fiction",
    "The Lady Of Blossholme": "Historical/Adventure, Fiction",
    "The Mahatma and the Hare": "Fantasy/Fable, Fiction",
    "The People Of The Mist": "Adventure/Fantasy, Fiction",
    "The Virgin of the Sun": "Historical/Adventure, Fiction",
    "The Wanderer's Necklace": "Historical/Adventure, Fiction",
    "The Wizard": "Adventure/Fantasy, Fiction",
    "The Yellow God": "Adventure/Fantasy, Fiction",
    "When the World Shook": "Science Fiction/Adventure, Fiction",
    "A Modern Utopia": "Science Fiction/Utopian, Fiction",
    "A Short History of the World": "History, Non-Fiction",
    "An Englishman Looks at the World": "Literature/Essays, Non-Fiction",
    "Ann Veronica": "Literature/Novel, Fiction",
    "Anticipations": "Science Fiction/Futurism, Non-Fiction",
    "Boon, The Mind of the Race, The Wild Asses of the Devil, and The Last Trump": "Literature/Satire, Fiction",
    "Certain Personal Matters": "Literature/Essays, Non-Fiction",
    "First and Last Things": "Philosophy, Non-Fiction",
    "Floor Games; a companion volume to _Little Wars_": "Games/Children's, Non-Fiction",
    "God The Invisible King": "Philosophy/Religion, Non-Fiction"
    "In the Days of the Comet": "Science Fiction, Fiction",
    "In The Fourth Year": "Politics/Essay, Non-Fiction",
    "Kipps": "Literature/Novel, Fiction",
    "Little Wars; a game for boys from twelve years of age to one hundred and fifty and for that more intelligent sort of girl who likes boys' games and books": "Games/War, Non-Fiction",
    "Love and Mr. Lewisham": "Literature/Novel, Fiction",
    "Mankind in the Making": "Philosophy/Social, Non-Fiction",
    "Marriage": "Literature/Novel, Fiction",
    "Mr. Britling Sees It Through": "Literature/War, Fiction",
    "New Worlds For Old": "Politics/Socialism, Non-Fiction",
    "Select Conversations with an Uncle": "Literature/Short Stories, Fiction",
    "Soul of a Bishop": "Literature/Novel, Fiction",
    "Tales of Space and Time": "Science Fiction/Short Stories, Fiction",
    "Text Book of Biology, Part 1_ Vertebrata": "Science/Biology, Non-Fiction",
    "The Country of the Blind, And Other Stories": "Science Fiction/Short Stories, Fiction",
    "The Door in the Wall And Other Stories": "Fantasy/Short Stories, Fiction",
    "The First Men In The Moon": "Science Fiction/Adventure, Fiction",
    "The Food of the Gods and How It Came to Earth": "Science Fiction, Fiction",
    "The History of Mr. Polly": "Literature/Humor, Fiction",
    "The Invisible Man": "Science Fiction/Horror, Fiction",
    "The Island of Doctor Moreau": "Science Fiction/Horror, Fiction",
    "The New Machiavelli": "Literature/Political, Fiction",
    "The Passionate Friends": "Literature/Novel, Fiction",
    "The Red Room": "Horror/Short Story, Fiction",
    "The Research Magnificent": "Literature/Novel, Fiction",
    "The Salvaging Of Civilisation": "Philosophy/Social, Non-Fiction",
    "The Sea Lady": "Fantasy/Romance, Fiction",
    "The Secret Places of the Heart": "Literature/Novel, Fiction",
    "The Sleeper Awakes": "Science Fiction/Dystopian, Fiction",
    "The Stolen Bacillus and Other Incidents": "Science Fiction/Short Stories, Fiction",
    "The Time Machine": "Science Fiction, Fiction",
    "The War in the Air": "Science Fiction/War, Fiction",
    "The War of the Worlds": "Science Fiction/Invasion, Fiction",
    "The Wheels of Chance": "Literature/Humor, Fiction",
    "The Wife of Sir Isaac Harman": "Literature/Novel, Fiction",
    "The Wonderful Visit": "Fantasy/Satire, Fiction",
    "The World Set Free": "Science Fiction, Fiction",
    "Tono Bungay": "Literature/Novel, Fiction",
    "Twelve Stories and a Dream": "Fantasy/Short Stories, Fiction",
    "War and the Future": "History/War, Non-Fiction",
    "What is Coming_": "Politics/Futurism, Non-Fiction",
    "When the Sleeper Wakes": "Science Fiction/Dystopian, Fiction",
    "Essays on Education and Kindred Subjects": "Education/Philosophy, Non-Fiction",
    "Essays_ Scientific, Political, & Speculative, Volume I": "Science/Essays, Non-Fiction",
    "The Philosophy of Style": "Language/Essay, Non-Fiction",
    "The Right To Ignore The State": "Politics/Anarchism, Non-Fiction",
    "Bartleby, The Scrivener": "Literature/Short Story, Fiction",
    "Battle-Pieces and Aspects of the War": "Poetry/War, Fiction",
    "I and My Chimney": "Literature/Short Story, Fiction",
    "Israel Potter": "Historical/Novel, Fiction",
    "John Marr and Other Poems": "Poetry, Fiction",
    "Mardi_ and A Voyage Thither, Volume 1": "Adventure/Novel, Fiction",
    "Mardi_ and A Voyage Thither, Volume 2": "Adventure/Novel, Fiction",
    "Moby Dick": "Adventure/Sea, Fiction",
    "Omoo_ Adventures in the South Seas": "Adventure/Travel, Fiction",
    "Pierre; or The Ambiguities": "Literature/Novel, Fiction",
    "Redburn. His First Voyage": "Adventure/Sea, Fiction",
    "The Confidence-Man": "Literature/Satire, Fiction",
    "The Piazza Tales": "Literature/Short Stories, Fiction",
    "Typee": "Adventure/Travel, Fiction",
    "White Jacket": "Adventure/Sea, Fiction",
    "Howard Pyle's Book of Pirates": "Adventure/Pirate, Fiction",
    "Men of Iron": "Historical/Adventure, Fiction",
    "Otto of the Silver Hand": "Historical/Adventure, Fiction",
    "Pepper & Salt": "Children's/Fairy Tales, Fiction",
    "Stolen Treasure": "Adventure/Short Stories, Fiction",
    "The Merry Adventures of Robin Hood": "Adventure/Folklore, Fiction",
    "The Rose of Paradise": "Adventure/Pirate, Fiction",
    "The Ruby of Kishmoor": "Adventure/Novella, Fiction",
    "The Story of Sir Launcelot and His Companions": "Adventure/Fantasy, Fiction",
    "The Story of the Champions of the Round Table": "Adventure/Fantasy, Fiction",
    "Twilight Land": "Fantasy/Fairy Tales, Fiction",
    "Youth": "Adventure/Short Story, Fiction",
    "A Daughter of the Snows": "Adventure/Novel, Fiction",
    "A Son Of The Sun": "Adventure/Short Stories, Fiction",
    "Adventure": "Adventure/Novel, Fiction",
    "Before Adam": "Science Fiction/Prehistoric, Fiction",
    "Brown Wolf and Other Jack London Stories": "Adventure/Short Stories, Fiction",
    "Burning Daylight": "Adventure/Western, Fiction",
    "Children of the Frost": "Adventure/Short Stories, Fiction",
    "Dutch Courage and Other Stories": "Adventure/Short Stories, Fiction",
    "Jerry of the Islands": "Adventure/Novel, Fiction",
    "John Barleycorn": "Biography/Memoir, Non-Fiction",
    "Lost Face": "Adventure/Short Stories, Fiction",
    "Love of Life and Other Stories": "Adventure/Short Stories, Fiction",
    "Martin Eden": "Literature/Novel, Fiction",
    "Michael, Brother of Jerry": "Adventure/Novel, Fiction",
    "Moon-Face and Other Stories": "Adventure/Short Stories, Fiction",
    "On the Makaloa Mat": "Adventure/Short Stories, Fiction",
    "Revolution and Other Essays": "Politics/Essays, Non-Fiction",
    "Smoke Bellew": "Adventure/Novel, Fiction",
    "South Sea Tales": "Adventure/Short Stories, Fiction",
    "Stories of Ships and the Sea": "Adventure/Short Stories, Fiction",
    "Tales of the Fish Patrol": "Adventure/Short Stories, Fiction",
    "The Acorn-Planter": "Drama, Fiction",
    "The Call of the Wild": "Adventure/Novel, Fiction",
    "The Cruise of the Dazzler": "Adventure/Sea, Fiction",
    "The Cruise of the Snark": "Travel/Adventure, Non-Fiction",
    "The Faith of Men": "Adventure/Short Stories, Fiction",
    "The Game": "Adventure/Novella, Fiction",
    "The God of His Fathers": "Adventure/Short Stories, Fiction",
    "The House of Pride": "Adventure/Short Stories, Fiction",
    "The Human Drift and Other Stories": "Adventure/Short Stories, Fiction",
    "The Iron Heel": "Science Fiction/Dystopian, Fiction",
    "The Jacket (The Star-Rover)": "Science Fiction/Adventure, Fiction",
    "The Little Lady of the Big House": "Literature/Novel, Fiction",
    "The Mutiny of the Elsinore": "Adventure/Sea, Fiction",
    "The Night-Born": "Adventure/Short Stories, Fiction",
    "The People of the Abyss": "Social/Documentary, Non-Fiction",
    "The Red One": "Science Fiction/Short Stories, Fiction",
    "The Road": "Biography/Travel, Non-Fiction",
    "The Scarlet Plague": "Science Fiction/Apocalyptic, Fiction",
    "The Sea-Wolf": "Adventure/Sea, Fiction",
    "The Son of the Wolf": "Adventure/Short Stories, Fiction",
    "The Strength of the Strong": "Adventure/Short Stories, Fiction",
    "The Turtles of Tasman": "Adventure/Short Stories, Fiction",
    "The Valley of the Moon": "Adventure/Novel, Fiction",
    "Theft": "Drama, Fiction",
    "War of the Classes": "Politics/Essays, Non-Fiction",
    "When God Laughs and Other Stories": "Adventure/Short Stories, Fiction",
    "White Fang": "Adventure/Novel, Fiction",
    "Alexander the Great": "Biography/History, Non-Fiction",
    "Caleb in the Country": "Children's/Religious, Fiction",
    "Charles I": "Biography/History, Non-Fiction",
    "Cyrus the Great": "Biography/History, Non-Fiction",
    "Darius the Great": "Biography/History, Non-Fiction",
    "Forests of Maine": "Children's/Nature, Non-Fiction",
    "Genghis Khan": "Biography/History, Non-Fiction",
    "Gentle Measures in the Management and Training of the Young": "Education/Parenting, Non-Fiction",
    "Georgie": "Children's/Story, Fiction",
    "Hannibal": "Biography/History, Non-Fiction",
    "History of Julius Caesar": "Biography/History, Non-Fiction",
    "History of King Charles II of England": "Biography/History, Non-Fiction",
    "Jonas on a Farm in Winter": "Children's/Story, Fiction",
    "King Alfred of England": "Biography/History, Non-Fiction",
    "Marco Paul's Voyages and Travels; Vermont": "Children's/Travel, Fiction",
    "Margaret of Anjou": "Biography/History, Non-Fiction",
    "Mary Erskine": "Children's/Story, Fiction",
    "Mary Queen of Scots": "Biography/History, Non-Fiction",
    "Nero": "Biography/History, Non-Fiction",
    "Peter the Great": "Biography/History, Non-Fiction",
    "Pyrrhus": "Biography/History, Non-Fiction",
    "Queen Elizabeth": "Biography/History, Non-Fiction",
    "Richard I": "Biography/History, Non-Fiction",
    "Richard II": "Biography/History, Non-Fiction",
    "Richard III": "Biography/History, Non-Fiction",
    "Rollo at Play": "Children's/Story, Fiction",
    "Rollo at Work": "Children's/Story, Fiction",
    "Rollo in Geneva": "Children's/Travel, Fiction",
    "Rollo in Holland": "Children's/Travel, Fiction",
    "Rollo in London": "Children's/Travel, Fiction",
    "Rollo in Naples": "Children's/Travel, Fiction",
    "Rollo in Paris": "Children's/Travel, Fiction",
    "Rollo in Rome": "Children's/Travel, Fiction",
    "Rollo in Scotland": "Children's/Travel, Fiction",
    "Rollo in Switzerland": "Children's/Travel, Fiction",
    "Rollo in the Woods": "Children's/Story, Fiction",
    "Rollo on the Atlantic": "Children's/Travel, Fiction",
    "Rollo on the Rhine": "Children's/Travel, Fiction",
    "Rollo's Experiments": "Children's/Science, Fiction",
    "Rollo's Museum": "Children's/Story, Fiction",
    "Rollo's Philosophy. [Air]": "Children's/Philosophy, Non-Fiction",
    "Romulus": "Biography/History, Non-Fiction",
    "Stuyvesant": "Children's/Story, Fiction",
    "The Teacher": "Education, Non-Fiction",
    "William the Conqueror": "Biography/History, Non-Fiction",
    "Xerxes": "Biography/History, Non-Fiction",
    "Goblin Tales of Lancashire": "Fantasy/Folklore, Fiction",
    "A Residence in France": "Travel/Memoir, Non-Fiction",
    "Afloat And Ashore": "Adventure/Sea, Fiction",
    "Autobiography of a Pocket-Hankerchief": "Literature/Short Story, Fiction",
    "Home as Found": "Literature/Novel, Fiction",
    "Homeward Bound": "Adventure/Sea, Fiction",
    "Jack Tier or The Florida Reef": "Adventure/Sea, Fiction",
    "Mercedes of Castile": "Historical/Adventure, Fiction",
    "Miles Wallingford": "Adventure/Sea, Fiction",
    "Ned Myers": "Biography/Sea, Non-Fiction",
    "New York": "History, Non-Fiction",
    "Oak Openings": "Adventure/Western, Fiction",
    "Precaution": "Literature/Novel, Fiction",
    "Recollections of Europe": "Travel/Memoir, Non-Fiction",
    "Satanstoe": "Historical/Novel, Fiction",
    "Tales for Fifteen": "Literature/Short Stories, Fiction",
    "The Bravo": "Historical/Adventure, Fiction",
    "The Chainbearer": "Historical/Novel, Fiction",
    "The Crater": "Adventure/Utopian, Fiction",
    "The Deerslayer": "Adventure/Western, Fiction",
    "The Headsman": "Historical/Novel, Fiction",
    "The Lake Gun": "Literature/Short Story, Fiction",
    "The Last of the Mohicans": "Historical/Adventure, Fiction",
    "The Monikins": "Satire/Fantasy, Fiction",
    "The Pathfinder": "Adventure/Western, Fiction",
    "The Pilot": "Adventure/Sea, Fiction",
    "The Pioneers": "Adventure/Western, Fiction",
    "The Prairie": "Adventure/Western, Fiction",
    "The Red Rover": "Adventure/Sea, Fiction",
    "The Redskins; or, Indian and Injin, Volume 1": "Adventure/Western, Fiction",
    "The Sea Lions": "Adventure/Sea, Fiction",
    "The Spy": "Historical/Adventure, Fiction",
    "The Two Admirals": "Adventure/Sea, Fiction",
    "The Water-Witch": "Adventure/Sea, Fiction",
    "The Wept of Wish-Ton-Wish": "Historical/Adventure, Fiction",
    "The Wing-and-Wing": "Adventure/Sea, Fiction",
    "Wyandotte": "Historical/Adventure, Fiction",
    "A Portrait of the Artist as a Young Man": "Literature/Novel, Fiction",
    "Chamber Music": "Poetry, Fiction",
    "Dubliners": "Literature/Short Stories, Fiction",
    "Ulysses": "Literature/Novel, Fiction",
    "A Holiday in Bed and Other Sketches": "Humor/Essays, Non-Fiction",
    "A Window in Thrums": "Literature/Novel, Fiction",
    "Alice Sit-By-The-Fire": "Drama/Comedy, Fiction",
    "An Edinburgh Eleven": "Biography/Sketches, Non-Fiction",
    "Auld Licht Idylls": "Literature/Short Stories, Fiction",
    "Better Dead": "Literature/Novella, Fiction",
    "Courage": "Philosophy/Essay, Non-Fiction",
    "Dear Brutus": "Drama/Fantasy, Fiction",
    "Echoes of the War": "Drama/War, Fiction",
    "Margaret Ogilvy": "Biography/Memoir, Non-Fiction",
    "My Lady Nicotine": "Humor/Essays, Fiction",
    "Peter and Wendy": "Children's/Fantasy, Fiction",
    "Peter Pan in Kensington Gardens, Version 1": "Children's/Fantasy, Fiction",
    "Peter Pan in Kensington Gardens, Version 2": "Children's/Fantasy, Fiction",
    "Peter Pan": "Children's/Fantasy, Fiction",
    "Quality Street": "Drama/Comedy, Fiction",
    "Sentimental Tommy": "Literature/Novel, Fiction",
    "The Admirable Crichton": "Drama/Comedy, Fiction",
    "The Little Minister": "Literature/Novel, Fiction",
    "The Little White Bird": "Children's/Fantasy, Fiction",
    "Tommy and Grizel": "Literature/Novel, Fiction",
    "What Every Woman Knows": "Drama/Comedy, Fiction",
    "_Der Tag_": "Drama/War, Fiction",
    "A District Messenger Boy and a Necktie Party": "Young Adult/Adventure, Fiction",
    "A Runaway Brig": "Young Adult/Adventure, Fiction",
    "An Amateur Fireman": "Young Adult/Adventure, Fiction",
    "Aunt Hannah and Seth": "Young Adult/Story, Fiction",
    "Commodore Barney's Young Spies": "Historical/Adventure, Fiction",
    "Corporal 'Lige's Recruit": "Historical/War, Fiction",
    "Defending the Island": "Young Adult/Adventure, Fiction",
    "Dick in the Desert": "Young Adult/Adventure, Fiction",
    "Down the Slope": "Young Adult/Adventure, Fiction",
    "Left Behind": "Young Adult/Adventure, Fiction",
    "Messenger No. 48": "Young Adult/Mystery, Fiction",
    "Mr. Stubbs's Brother": "Young Adult/Adventure, Fiction",
    "Neal, the Miller": "Historical/Story, Fiction",
    "On the Kentucky Frontier": "Historical/Adventure, Fiction",
    "Ralph Gurney's Oil Speculation": "Young Adult/Adventure, Fiction",
    "Richard of Jamestown": "Historical/Adventure, Fiction",
    "Teddy and Carrots": "Young Adult/Adventure, Fiction",
    "The Adventures of a Country Boy at a Country Fair": "Young Adult/Adventure, Fiction",
    "The Boys of ’98": "Historical/War, Fiction",
    "The Club at Crow's Corner": "Young Adult/Adventure, Fiction",
    "The Minute Boys of Boston": "Historical/Adventure, Fiction",
    "The Minute Boys of the Mohawk Valley": "Historical/Adventure, Fiction",
    "The Minute Boys of York Town": "Historical/Adventure, Fiction",
    "The Princess and Joe Potter": "Young Adult/Adventure, Fiction",
    "The Search for the Silver City": "Young Adult/Adventure, Fiction",
    "Toby Tyler": "Young Adult/Adventure, Fiction",
    "Under the Liberty Tree": "Historical/Adventure, Fiction",
    "Abraham Lincoln": "Biography/History, Non-Fiction",
    "Among My Books, First Series": "Literature/Essays, Non-Fiction",
    "Among My Books, Second Series": "Literature/Essays, Non-Fiction",
    "My Garden Acquaintance": "Nature/Essay, Non-Fiction",
    "Poems of James Russell Lowell": "Poetry, Fiction",
    "The Biglow Papers": "Poetry/Satire, Fiction",
    "The Complete Poetical Works of James Russell Lowell": "Poetry, Fiction",
    "The Function Of The Poet And Other Essays": "Literature/Essays, Non-Fiction",
    "The Vision of Sir Launfal And Other Poems, Version 1": "Poetry, Fiction",
    "The Vision of Sir Launfal And Other Poems, Version 2": "Poetry, Fiction",
    "The Writings of James Russell Lowell in Prose and Poetry, Volume V": "Literature/Collection, Fiction",
    "Emma": "Literature/Romance, Fiction",
    "Lady Susan": "Literature/Novella, Fiction",
    "Love And Freindship And Other Early Works": "Literature/Short Stories, Fiction",
    "Mansfield Park": "Literature/Novel, Fiction",
    "Northanger Abbey": "Literature/Gothic, Fiction",
    "Persuasion": "Literature/Romance, Fiction",
    "Pride and Prejudice": "Literature/Romance, Fiction",
    "Sense and Sensibility": "Literature/Romance, Fiction",
    "All Roads Lead to Calvary": "Literature/Novel, Fiction",
    "Clocks": "Humor/Short Story, Fiction",
    "Diary of a Pilgrimage": "Travel/Humor, Fiction",
    "Dreams": "Philosophy/Essay, Non-Fiction",
    "Evergreens": "Humor/Essay, Non-Fiction",
    "Fanny and the Servant Problem": "Drama/Comedy, Fiction",
    "Idle Ideas in 1905": "Humor/Essays, Non-Fiction",
    "Idle Thoughts of an Idle Fellow": "Humor/Essays, Non-Fiction",
    "John Ingerfield and Other Stories": "Literature/Short Stories, Fiction",
    "Malvina of Brittany": "Fantasy/Short Stories, Fiction",
    "Mrs. Korner Sins Her Mercies": "Humor/Short Story, Fiction",
    "Novel Notes": "Humor/Novel, Fiction",
    "Passing of the Third Floor Back": "Fantasy/Drama, Fiction",
    "Paul Kelver": "Literature/Novel, Fiction",
    "Sketches in Lavender, Blue and Green": "Literature/Short Stories, Fiction",
    "Stage-Land": "Humor/Theater, Non-Fiction",
    "Tea-table Talk": "Humor/Essays, Non-Fiction",
    "The Angel and the Author - and Others": "Humor/Short Stories, Fiction",
    "The Cost of Kindness": "Humor/Short Story, Fiction",
    "The Love of Ulrich Nebendahl": "Fantasy/Short Story, Fiction",
    "The Master of Mrs. Chilvers": "Drama/Social, Fiction",
    "The Observations of Henry": "Humor/Short Stories, Fiction",
    "The Philosopher's Joke": "Fantasy/Short Story, Fiction",
    "The Second Thoughts of An Idle Fellow": "Humor/Essays, Non-Fiction",
    "The Soul of Nicholas Snyders": "Fantasy/Short Story, Fiction",
    "They and I": "Literature/Novel, Fiction",
    "Three Men in a Boat": "Humor/Travel, Fiction",
    "Three Men on the Bummel": "Humor/Travel, Fiction",
    "Told After Supper": "Humor/Ghost Stories, Fiction",
    "Tommy and Co": "Humor/Novel, Fiction",
    "Grace Abounding to the Chief of Sinners": "Biography/Religious, Non-Fiction",
    "Miscellaneous Pieces": "Religion/Essays, Non-Fiction",
    "The Heavenly Footman": "Religion/Allegory, Non-Fiction",
    "The Holy War": "Religion/Allegory, Fiction",
    "The Jerusalem Sinner Saved": "Religion/Sermon, Non-Fiction",
    "The Life and Death of Mr. Badman": "Religion/Allegory, Fiction",
    "The Pharisee And The Publican": "Religion/Parable, Non-Fiction",
    "The Pilgrim's Progress": "Religion/Allegory, Fiction",
    "The Works of John Bunyan": "Religion/Collection, Non-Fiction",
    "All for Love": "Drama/Tragedy, Fiction",
    "Discourses on Satire and Epic Poetry": "Literature/Criticism, Non-Fiction",
    "Dryden's Works Vol. 16 (of 18)": "Literature/Collection, Fiction",
    "Dryden's Works Vol. 2 (of 18)": "Literature/Collection, Fiction",
    "Dryden's Works Vol. 3 (of 18)": "Literature/Collection, Fiction",
    "Dryden's Works Vol. 4 (of 18)": "Literature/Collection, Fiction",
    "Dryden's Works Vol. 5 (of 18)": "Literature/Collection, Fiction",
    "Dryden's Works Vol. 6 (of 18)": "Literature/Collection, Fiction",
    "Dryden's Works Vol. 7 (of 18)": "Literature/Collection, Fiction",
    "His Majesties Declaration Defended": "Politics/Defense, Non-Fiction",
    "Palamon and Arcite": "Poetry, Fiction",
    "The Poetical Works of John Dryden, Volume 1": "Poetry, Fiction",
    "The Poetical Works of John Dryden, Volume 2": "Poetry, Fiction",
    "A Bit O' Love": "Drama, Fiction",
    "A Family Man": "Drama, Fiction",
    "Another Sheaf": "Literature/Essays, Non-Fiction",
    "Beyond": "Literature/Novel, Fiction",
    "Essays Concerning Letters": "Literature/Essays, Non-Fiction",
    "Essays on Censorship and Art": "Literature/Essays, Non-Fiction",
    "Five Tales": "Literature/Short Stories, Fiction",
    "Fraternity": "Literature/Novel, Fiction",
    "Hall-Marked and Others": "Drama, Fiction",
    "Inn of Tranquility and Other Essays": "Literature/Essays, Non-Fiction",
    "Joy": "Drama, Fiction",
    "Justice": "Drama, Fiction",
    "Loyalties": "Drama, Fiction",
    "Quality and Other Studies and Essays": "Literature/Essays, Non-Fiction",
    "Saint's Progress": "Literature/Novel, Fiction",
    "Six Short Plays, Complete": "Drama, Fiction",
    "Strife": "Drama, Fiction",
    "Tatterdemalion": "Literature/Short Stories, Fiction",
    "The Burning Spear": "Literature/Satire, Fiction",
    "The Complete Essays of John Galsworthy": "Literature/Essays, Non-Fiction",
    "The Country House": "Literature/Novel, Fiction",
    "The Dark Flower": "Literature/Novel, Fiction",
    "The Eldest Son": "Drama, Fiction",
    "The First and The Last": "Literature/Short Story, Fiction",
    "The Forsyte Saga, Complete": "Literature/Novel, Fiction",
    "The Foundations": "Drama, Fiction",
    "The Freelands": "Literature/Novel, Fiction",
    "The Fugitive": "Drama, Fiction",
    "The Island Pharisees": "Literature/Novel, Fiction",
    "The Little Dream": "Drama, Fiction",
    "The Little Man": "Drama, Fiction",
    "The Mob": "Drama, Fiction",
    "The Patrician": "Literature/Novel, Fiction",
    "The Pigeon": "Drama, Fiction",
    "The Project Gutenberg Plays of John Galsworthy, Complete": "Drama/Collection, Fiction",
    "The Silver Box": "Drama, Fiction",
    "The Skin Game": "Drama, Fiction",
    "To Let": "Literature/Novel, Fiction",
    "Villa Rubein and Other Stories": "Literature/Short Stories, Fiction",
    "Windows": "Drama, Fiction",
    "Keats_ Poems Published in 1820": "Poetry, Fiction",
    "Lamia": "Poetry, Fiction",
    "Letters of John Keats to His Family and Friends": "Biography/Letters, Non-Fiction",
    "Poems 1817": "Poetry, Fiction",
    "An Essay Concerning Humane Understanding, Volume 1": "Philosophy, Non-Fiction",
    "An Essay Concerning Humane Understanding, Volume 2": "Philosophy, Non-Fiction",
    "Second Treatise of Government": "Politics/Philosophy, Non-Fiction",
    "The Economic Consequences of the Peace": "Economics/Politics, Non-Fiction",
    "Areopagitica": "Politics/Essay, Non-Fiction",
    "L'Allegro, Il Penseroso, Comus, and Lycidas": "Poetry, Fiction",
    "Milton's Comus": "Poetry/Drama, Fiction",
    "Minor Poems by Milton": "Poetry, Fiction",
    "Paradise Lost": "Poetry/Epic, Fiction",
    "Paradise Regained": "Poetry/Epic, Fiction",
    "The Poetical Works of John Milton": "Poetry, Fiction",
    "Burke": "Biography/Politics, Non-Fiction",
    "Critical Miscellanies, Volume 1, Essay 1, Robespierre": "History/Essay, Non-Fiction",
    "Critical Miscellanies, Volume 1, Essay 2, Carlyle": "Literature/Essay, Non-Fiction",
    "Critical Miscellanies, Volume 1, Essay 3, Byron": "Literature/Essay, Non-Fiction",
    "Critical Miscellanies, Volume 1, Essay 4, Macaulay": "Literature/Essay, Non-Fiction",
    "Critical Miscellanies, Volume 1, Essay 5, Emerson": "Literature/Essay, Non-Fiction",
    "Critical Miscellanies, Volume 2, Essay 1, Vauvenargues": "Philosophy/Essay, Non-Fiction",
    "Critical Miscellanies, Volume 2, Essay 2, Turgot": "Biography/Essay, Non-Fiction",
    "Critical Miscellanies, Volume 2, Essay 3, Condorcet": "Philosophy/Essay, Non-Fiction",
    "Critical Miscellanies, Volume 2, Essay 4, Joseph de Maistre": "Philosophy/Essay, Non-Fiction",
    "Critical Miscellanies, Volume 3, Essay 1, On Popular Culture": "Social/Essay, Non-Fiction",
    "Critical Miscellanies, Volume 3, Essay 10, Auguste Comte": "Philosophy/Essay, Non-Fiction",
    "Critical Miscellanies, Volume 3, Essay 4, The Life of George Eliot": "Biography/Essay, Non-Fiction",
    "Critical Miscellanies, Volume 3, Essay 5, On Pattison's Memoirs": "Literature/Essay, Non-Fiction",
    "Critical Miscellanies, Volume 3, Essay 6, Harriet Martineau": "Biography/Essay, Non-Fiction",
    "Critical Miscellanies, Volume 3, Essay 7, A Sketch": "Literature/Essay, Non-Fiction",
    "Critical Miscellanies, Volume 3, Essay 8, France in the Eighteenth Century": "History/Essay, Non-Fiction",
    "Critical Miscellanies, Volume 3, Essay 9, The Expansion of England": "History/Essay, Non-Fiction",
    "Critical Miscellanies, Volume 3, Essays 2 and 3": "Literature/Essays, Non-Fiction",
    "Diderot and the Encyclopædists (Vol 1 of 2)": "Biography/Philosophy, Non-Fiction",
    "Diderot and the Encyclopædists (Vol 2 of 2)": "Biography/Philosophy, Non-Fiction",
    "Indian speeches (1907-1909)": "Politics/Speeches, Non-Fiction",
    "On Compromise": "Philosophy/Essay, Non-Fiction",
    "Rousseau": "Biography/Philosophy, Non-Fiction",
    "Studies in Literature": "Literature/Criticism, Non-Fiction",
    "The Life of William Ewart Gladstone (Vol 1 of 3)": "Biography/Politics, Non-Fiction",
    "The Life of William Ewart Gladstone (Vol 2 of 3)": "Biography/Politics, Non-Fiction",
    "The Life of William Ewart Gladstone (Vol 3 of 3)": "Biography/Politics, Non-Fiction",
    "A Joy For Ever": "Art/Economics, Non-Fiction",
    "Aratra Pentelici, Seven Lectures on the Elements of Sculpture": "Art/Lectures, Non-Fiction",
    "Ariadne Florentina": "Art/Lectures, Non-Fiction",
    "Frondes Agrestes": "Nature/Essays, Non-Fiction",
    "Giotto and his works in Padua": "Art/Biography, Non-Fiction",
    "Hortus Inclusus": "Nature/Letters, Non-Fiction",
    "Lectures on Architecture and Painting": "Art/Lectures, Non-Fiction",
    "Lectures on Art": "Art/Lectures, Non-Fiction",
    "Lectures on Landscape": "Art/Lectures, Non-Fiction",
    "Letters to the Clergy": "Religion/Letters, Non-Fiction",
    "Love's Meinie": "Nature/Lectures, Non-Fiction",
    "Modern Painters, Volume 1 (of 5)": "Art/Criticism, Non-Fiction",
    "Modern Painters, Volume 2 (of 5)": "Art/Criticism, Non-Fiction",
    "Modern Painters, Volume 3 (of 5)": "Art/Criticism, Non-Fiction",
    "Modern Painters, Volume 4 (of 5)": "Art/Criticism, Non-Fiction",
    "Mornings in Florence": "Art/Travel, Non-Fiction",
    "On the Old Road, Vol. 1 (of 2)": "Art/Essays, Non-Fiction",
    "On the Old Road, Vol. 2 (of 2)": "Art/Essays, Non-Fiction",
    "Our Fathers Have Told Us, Part I, The Bible of Amiens": "Religion/Art, Non-Fiction",
    "Proserpina, Volume 1": "Nature/Botany, Non-Fiction",
    "Proserpina, Volume 2": "Nature/Botany, Non-Fiction",
    "Saint Ursula": "Religion/Biography, Non-Fiction",
    "Selections From the Works of John Ruskin": "Art/Collection, Non-Fiction",
    "Sesame and Lilies": "Education/Lectures, Non-Fiction",
    "Stones of Venice [introductions]": "Art/Architecture, Non-Fiction",
    "The Crown of Wild Olive": "Art/Lectures, Non-Fiction",
    "The Elements of Drawing": "Art/Instruction, Non-Fiction",
    "The Ethics of the Dust": "Philosophy/Lectures, Non-Fiction",
    "The Harbours of England": "Art/Travel, Non-Fiction",
    "The King of the Golden River": "Children's/Fairy Tale, Fiction",
    "The Pleasures of England": "History/Lectures, Non-Fiction",
    "The Poetry of Architecture": "Art/Architecture, Non-Fiction",
    "The Queen of the Air": "Mythology/Lectures, Non-Fiction",
    "The Seven Lamps of Architecture": "Art/Architecture, Non-Fiction",
    "The Stones of Venice, Volume I (of 3)": "Art/Architecture, Non-Fiction",
    "The Stones of Venice, Volume II (of 3)": "Art/Architecture, Non-Fiction",
    "The Stones of Venice, Volume III (of 3)": "Art/Architecture, Non-Fiction",
    "The Storm-Cloud of the Nineteenth Century": "Nature/Lectures, Non-Fiction",
    "The Two Paths": "Art/Lectures, Non-Fiction",
    "Time and Tide by Weare and Tyne": "Social/Letters, Non-Fiction",
    "Unto This Last and Other Essays on Political Economy": "Economics/Essays, Non-Fiction",
    "Val d'Arno": "Art/Lectures, Non-Fiction",
    "A System Of Logic, Ratiocinative And Inductive": "Philosophy/Logic, Non-Fiction",
    "Auguste Comte and Positivism": "Philosophy, Non-Fiction",
    "Autobiography": "Biography, Non-Fiction",
    "Considerations on Representative Government": "Politics, Non-Fiction",
    "Essays on some unsettled Questions of Political Economy": "Economics/Essays, Non-Fiction",
    "On Liberty": "Philosophy/Politics, Non-Fiction",
    "Principles Of Political Economy": "Economics, Non-Fiction",
    "Socialism": "Politics/Economics, Non-Fiction",
    "The Contest in America": "Politics/Essay, Non-Fiction",
    "The Subjection of Women": "Social/Feminism, Non-Fiction",
    "Utilitarianism": "Philosophy, Non-Fiction",
    "A Modest Proposal": "Satire/Essay, Non-Fiction",
    "A Tale of a Tub": "Satire/Religion, Fiction",
    "Gulliver's Travels": "Satire/Adventure, Fiction",
    "The Battle of the Books": "Satire/Literature, Fiction",
    "The Bickerstaff-Partridge Papers": "Satire, Fiction",
    "The Journal to Stella": "Biography/Letters, Non-Fiction",
    "The Poems of Jonathan Swift, D.D., Volume 1": "Poetry, Fiction",
    "The Poems of Jonathan Swift, D.D., Volume 2": "Poetry, Fiction",
    "The Prose Works of Jonathan Swift, D.D., Volume 10": "Literature/Collection, Non-Fiction",
    "The Prose Works of Jonathan Swift, D.D., Volume 3": "Literature/Collection, Non-Fiction",
    "The Prose Works of Jonathan Swift, D.D., Volume 4": "Literature/Collection, Non-Fiction",
    "The Prose Works of Jonathan Swift, D.D., Volume 6": "Literature/Collection, Non-Fiction",
    "The Prose Works of Jonathan Swift, D.D., Volume 7": "Literature/Collection, Non-Fiction",
    "The Prose Works of Jonathan Swift, D.D., Volume 9": "Literature/Collection, Non-Fiction",
    "Three Sermons, Three Prayers": "Religion/Sermons, Non-Fiction",
    "'Twixt Land & Sea": "Adventure/Short Stories, Fiction",
    "A Personal Record": "Biography/Memoir, Non-Fiction",
    "A Set of Six": "Adventure/Short Stories, Fiction",
    "Almayer's Folly": "Adventure/Novel, Fiction",
    "Amy Foster": "Adventure/Short Story, Fiction",
    "An Outcast of the Islands": "Adventure/Novel, Fiction",
    "Chance": "Adventure/Sea, Fiction",
    "End of the Tether": "Adventure/Novella, Fiction",
    "Falk": "Adventure/Novella, Fiction",
    "Gaspar Ruiz": "Adventure/Short Story, Fiction",
    "Heart of Darkness, Version 1": "Adventure/Novella, Fiction",
    "Heart of Darkness, Version 2": "Adventure/Novella, Fiction",
    "Lord Jim": "Adventure/Sea, Fiction",
    "Nostromo, A Tale of the Seaboard": "Adventure/Novel, Fiction",
    "Notes on Life and Letters": "Literature/Essays, Non-Fiction",
    "Notes on My Books": "Literature/Essays, Non-Fiction",
    "One Day More": "Drama, Fiction",
    "Some Reminiscences": "Biography/Memoir, Non-Fiction",
    "Tales Of Hearsay": "Adventure/Short Stories, Fiction",
    "Tales of Unrest": "Adventure/Short Stories, Fiction",
    "The Arrow of Gold": "Adventure/Novel, Fiction",
    "The Mirror of the Sea": "Biography/Sea, Non-Fiction",
    "The Nigger Of The _Narcissus_": "Adventure/Sea, Fiction",
    "The Point Of Honor": "Adventure/Novella, Fiction",
    "The Rescue": "Adventure/Novel, Fiction",
    "The Secret Agent": "Mystery/Spy, Fiction",
    "The Secret Sharer": "Adventure/Short Story, Fiction",
    "The Shadow-Line": "Adventure/Novella, Fiction",
    "To-morrow": "Adventure/Short Story, Fiction",
    "Typhoon": "Adventure/Novella, Fiction",
    "Under Western Eyes": "Literature/Political, Fiction",
    "Victory": "Adventure/Novel, Fiction",
    "Within the Tides": "Adventure/Short Stories, Fiction",
    "Captain Sword and Captain Pen": "Poetry/War, Fiction",
    "Stories from the Italian Poets_ With Lives of the Writers, Volume 1": "Poetry/Biography, Non-Fiction",
    "Stories from the Italian Poets_ With Lives of the Writers, Volume 2": "Poetry/Biography, Non-Fiction",
    "A Tangled Tale": "Children's/Mathematics, Fiction",
    "Alice's Adventures in Wonderland": "Children's/Fantasy, Fiction",
    "Alice's Adventures Under Ground": "Children's/Fantasy, Fiction",
    "Eight or Nine Wise Words about Letter-Writing": "Humor/Essay, Non-Fiction",
    "Feeding the Mind": "Philosophy/Essay, Non-Fiction",
    "Phantasmagoria and Other Poems": "Poetry/Humor, Fiction",
    "Rhyme_ And Reason_": "Poetry/Humor, Fiction",
    "Songs From Alice in Wonderland and Through the Looking-Glass": "Poetry/Children's, Fiction",
    "Sylvie and Bruno": "Children's/Fantasy, Fiction",
    "Symbolic Logic": "Mathematics/Logic, Non-Fiction",
    "The Game of Logic": "Mathematics/Logic, Non-Fiction",
    "The Hunting of the Snark": "Poetry/Humor, Fiction",
    "Three Sunsets and Other Poems": "Poetry, Fiction",
    "Through the Looking-Glass": "Children's/Fantasy, Fiction",
    "Byron's Poetical Works, Vol. 1": "Poetry, Fiction",
    "Childe Harold's Pilgrimage": "Poetry/Epic, Fiction",
    "Don Juan": "Poetry/Satire, Fiction",
    "Fugitive Pieces": "Poetry, Fiction",
    "The Works of Lord Byron. Vol. 2": "Poetry/Collection, Fiction",
    "The Works of Lord Byron. Vol. 3": "Poetry/Collection, Fiction",
    "The Works of Lord Byron. Vol. 4": "Poetry/Collection, Fiction",
    "The Works of Lord Byron. Vol. 5": "Poetry/Collection, Fiction",
    "The Works of Lord Byron. Vol. 6": "Poetry/Collection, Fiction",
    "The Works of Lord Byron. Vol. 7": "Poetry/Collection, Fiction",
    "The Works of Lord Byron_ Letters and Journals. Vol. 1": "Biography/Letters, Non-Fiction",
    "The Works of Lord Byron_ Letters and Journals. Vol. 2": "Biography/Letters, Non-Fiction",
    "Beauties of Tennyson": "Poetry, Fiction",
    "Becket and other plays": "Drama/Historical, Fiction",
    "Enoch Arden, &c": "Poetry, Fiction",
    "Idylls of the King": "Poetry/Epic, Fiction",
    "Lady Clare": "Poetry, Fiction",
    "Queen Mary and Harold": "Drama/Historical, Fiction",
    "The Early Poems of Alfred Lord Tennyson": "Poetry, Fiction",
    "The Last Tournament": "Poetry, Fiction",
    "The Princess": "Poetry, Fiction",
    "The Suppressed Poems of Alfred Lord Tennyson": "Poetry, Fiction",
    "A Garland for Girls": "Young Adult/Short Stories, Fiction",
    "A Modern Cinderella": "Young Adult/Short Stories, Fiction",
    "An Old-fashioned Girl": "Young Adult/Novel, Fiction",
    "Aunt Jo's Scrap-Bag VI": "Young Adult/Short Stories, Fiction",
    "Aunt Jo's Scrap-Bag, Volume 5": "Young Adult/Short Stories, Fiction",
    "Aunt Jo's Scrap-Bag": "Young Adult/Short Stories, Fiction",
    "Behind A Mask, Or A Woman's Power": "Mystery/Thriller, Fiction",
    "Comic Tragedies": "Drama/Tragedy, Fiction",
    "Eight Cousins": "Young Adult/Novel, Fiction",
    "Flower Fables": "Children's/Fairy Tales, Fiction",
    "Hospital Sketches": "Biography/War, Non-Fiction",
    "Jack and Jill": "Young Adult/Novel, Fiction",
    "Jo's Boys": "Young Adult/Novel, Fiction",
    "Kitty's Class Day And Other Stories": "Young Adult/Short Stories, Fiction",
    "Little Men": "Young Adult/Novel, Fiction",
    "Little Women": "Young Adult/Novel, Fiction",
    "Lulu's Library, Volume II": "Children's/Short Stories, Fiction",
    "Marjorie's Three Gifts": "Young Adult/Short Stories, Fiction",
    "May Flowers": "Young Adult/Short Stories, Fiction",
    "Moods": "Literature/Novel, Fiction",
    "Mountain-Laurel and Maidenhair": "Young Adult/Short Stories, Fiction",
    "On Picket Duty and Other Tales": "Literature/Short Stories, Fiction",
    "Pauline's Passion and Punishment": "Literature/Short Story, Fiction",
    "Rose in Bloom": "Young Adult/Novel, Fiction",
    "Shawl-Straps": "Young Adult/Travel, Fiction",
    "Silver Pitchers_ and Independence": "Young Adult/Short Stories, Fiction",
    "Spinning-Wheel Stories": "Young Adult/Short Stories, Fiction",
    "The Abbot's Ghost, Or Maurice Treherne's Temptation": "Mystery/Gothic, Fiction",
    "The Candy Country": "Children's/Fantasy, Fiction",
    "The Louisa Alcott Reader": "Children's/Stories, Fiction",
    "The Mysterious Key And What It Opened": "Mystery/Short Story, Fiction",
    "Three Unpublished Poems": "Poetry, Fiction",
    "Under the Lilacs": "Young Adult/Novel, Fiction",
    "Work_ A Story of Experience": "Literature/Novel, Fiction",
    "Anne Of Avonlea": "Young Adult/Novel, Fiction",
    "Anne Of Green Gables": "Young Adult/Novel, Fiction",
    "Anne Of The Island": "Young Adult/Novel, Fiction",
    "Anne's House of Dreams": "Young Adult/Novel, Fiction",
    "Chronicles of Avonlea": "Young Adult/Short Stories, Fiction",
    "Further Chronicles of Avonlea": "Young Adult/Short Stories, Fiction",
    "Kilmeny of the Orchard": "Young Adult/Romance, Fiction",
    "Lucy Maud Montgomery Short Stories, 1896 to 1901": "Literature/Short Stories, Fiction",
    "Lucy Maud Montgomery Short Stories, 1902 to 1903": "Literature/Short Stories, Fiction",
    "Lucy Maud Montgomery Short Stories, 1904": "Literature/Short Stories, Fiction",
    "Lucy Maud Montgomery Short Stories, 1905 to 1906": "Literature/Short Stories, Fiction",
    "Lucy Maud Montgomery Short Stories, 1907 to 1908": "Literature/Short Stories, Fiction",
    "Lucy Maud Montgomery Short Stories, 1909 to 1922": "Literature/Short Stories, Fiction",
    "Rainbow Valley": "Young Adult/Novel, Fiction",
    "Rilla of Ingleside": "Young Adult/Novel, Fiction"
}

